# Imports

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

import os
from datetime import datetime, timedelta

In [2]:
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Algorithms

In [4]:
def MLP(X_train, X_test, y_train, y_test, layers_config,
        regularizer=tf.keras.regularizers.l2, lambda_=0.1, task='classification',
        epochs=100, batch_size = 32):
    # Multi Layer Perceptron
    # layers_config expects a list of tuples (n_neurons, activation) [(10, 'relu'}]
    
    model = Sequential(
    [ 
        tf.keras.Input(shape=(X_train.shape[1],))]\
        + [
            Dense(layer_config[0], activation = layer_config[1], name='L'+str(l+1), kernel_regularizer=regularizer(lambda_)) \
            for l, layer_config in enumerate(layers_config[:-1])
        ]
        + [
            Dense(layers_config[-1][0], activation = layers_config[-1][1], name='L'+str(len(layers_config)))    # < softmax activation here
    ]
    )
    
    if task == 'classification':
        loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    elif task == 'regression':
        loss = tf.keras.losses.mean_squared_error(from_logits=True)
        
    model.compile(
        loss = loss,
        optimizer=tf.keras.optimizers.Adam(0.001),
    )
    
    if batch_size == -1:
        model.fit(X_train, y_train, epochs=epochs, batch_size=X_train.shape[0])
    else:
        model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)
    
    # it's more accurate mathematically to use linear function on last layer and then use softmax on the prediction
    if (layers_config[-1][1] == 'linear') & (task == 'classification'):
        y_train_pred_probs = tf.nn.softmax(model.predict(X_train)).numpy()
        y_test_pred_probs = tf.nn.softmax(model.predict(X_test)).numpy()
    else:
        y_train_pred_probs = model.predict(X_train)
        y_test_pred_probs = model.predict(X_test)

    y_train_pred = np.argmax(y_train_pred_probs, axis=1)
    y_test_pred = np.argmax(y_test_pred_probs, axis=1)
    
    return model, y_train_pred_probs, y_test_pred_probs, y_train_pred, y_test_pred

In [5]:
def GBT(X_train, X_test, y_train, y_test,
        max_depth=3, n_iter_no_change=None, verbose=0):
    
    model = GradientBoostingClassifier(max_depth=max_depth, n_iter_no_change=n_iter_no_change, verbose=verbose)\
        .fit(X_train, y_train)
    
    y_train_pred_probs = model.predict_proba(X_train)
    y_test_pred_probs = model.predict_proba(X_test)
    
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    return model, y_train_pred_probs, y_test_pred_probs, y_train_pred, y_test_pred

In [6]:
def RandomForest(X_train, X_test, y_train, y_test, verbose=0):
    
    model = RandomForestClassifier(verbose=verbose)\
        .fit(X_train, y_train)
    
    y_train_pred_probs = model.predict_proba(X_train)
    y_test_pred_probs = model.predict_proba(X_test)
    
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    return model, y_train_pred_probs, y_test_pred_probs, y_train_pred, y_test_pred

## Utils

In [7]:
def accuracy(confusion_matrix):
    diagonal_sum = confusion_matrix.trace()
    sum_of_all_elements = confusion_matrix.sum()
    return diagonal_sum / sum_of_all_elements

# Variables

In [8]:
footballdata_wTeamStats_path = \
    os.path.abspath(os.getcwd())+"\\Data\\teamStats_byLeague"

In [9]:
for filename in [f for f in os.listdir(footballdata_wTeamStats_path)]:
    print(filename)

footballdata_wTeamStats_Belgium_Jupiler.csv
footballdata_wTeamStats_England_Championship.csv
footballdata_wTeamStats_englishpremierleague.csv
footballdata_wTeamStats_France_Ligue1.csv
footballdata_wTeamStats_France_Ligue2.csv
footballdata_wTeamStats_Germany_Bundesliga1.csv
footballdata_wTeamStats_Germany_Bundesliga2.csv
footballdata_wTeamStats_Greece_SuperLeague.csv
footballdata_wTeamStats_Italy_SerieA.csv
footballdata_wTeamStats_Italy_SerieB.csv
footballdata_wTeamStats_Netherlands_Eredivisie.csv
footballdata_wTeamStats_Portugal_PrimeiraLiga.csv
footballdata_wTeamStats_Spain_Laliga.csv
footballdata_wTeamStats_Spain_Laliga2.csv
footballdata_wTeamStats_Turkey_SuperLeague.csv


In [10]:
leagues_toConsider = ['Netherlands_Eredivisie', 'Belgium_Jupiler']

In [11]:
[f for f in os.listdir(footballdata_wTeamStats_path) if any(x in f for x in leagues_toConsider)]

['footballdata_wTeamStats_Belgium_Jupiler.csv',
 'footballdata_wTeamStats_Netherlands_Eredivisie.csv']

# Load Football-Data File with Indexes

In [12]:
footballdata_wTeamStats = pd.DataFrame()
for filename in [f for f in os.listdir(footballdata_wTeamStats_path) if any(x in f for x in leagues_toConsider)]:
    footballdata_wTeamStats = footballdata_wTeamStats.append(pd.read_csv(footballdata_wTeamStats_path+"\\"+filename, 
                                                                         index_col=0))
footballdata_wTeamStats

,AverageAwayWinOdd,AverageDrawWinOdd,AverageHomeWinOdd,AvgOver2_5,AvgUnder2_5,AwayGoals,AwayTeamId,AwayTeamName,Date,Div,FullTimeResult,HomeGoals,HomeTeamName,HomeTeamid,Season,away_average_away_goals_against,away_average_away_goals_for,away_average_home_goals_against,away_average_home_goals_for,away_average_total_goals_against,away_average_total_goals_for,away_away_clean_sheets,away_away_draws,away_away_goals_against,away_away_goals_for,away_away_loses,away_away_wins,away_biggest_away_goals_against,away_biggest_away_goals_for,away_biggest_away_lose,away_biggest_away_win,away_biggest_draw_streak,away_biggest_home_goals_against,away_biggest_home_goals_for,away_biggest_home_lose,away_biggest_home_win,away_biggest_lose_streak,away_biggest_win_streak,away_failed_to_score_away,away_failed_to_score_home,away_failed_to_score_total,away_form,away_goals_against_minute_0_15,away_goals_against_minute_0_15_percentage,away_goals_against_minute_16_30,away_goals_against_minute_16_30_percentage,away_goals_against_minute_31_45,away_goals_against_minute_31_45_percentage,away_goals_against_minute_46_60,away_goals_against_minute_46_60_percentage,away_goals_against_minute_61_75,away_goals_against_minute_61_75_percentage,away_goals_against_minute_76_90,away_goals_against_minute_76_90_percentage,away_goals_against_minute_91_105,away_goals_against_minute_91_105_percentage,away_goals_for_minute_0_15,away_goals_for_minute_0_15_percentage,away_goals_for_minute_16_30,away_goals_for_minute_16_30_percentage,away_goals_for_minute_31_45,away_goals_for_minute_31_45_percentage,away_goals_for_minute_46_60,away_goals_for_minute_46_60_percentage,away_goals_for_minute_61_75,away_goals_for_minute_61_75_percentage,away_goals_for_minute_76_90,away_goals_for_minute_76_90_percentage,away_goals_for_minute_91_105,away_goals_for_minute_91_105_percentage,away_home_clean_sheets,away_home_draws,away_home_goals_against,away_home_goals_for,away_home_loses,away_home_wins,away_lineups,away_penalties_for,away_penalties_missed,away_penalties_missed_percentage,away_penalties_scored,away_penalties_scored_percentage,away_red_cards_minute_0_15,away_red_cards_minute_0_15_percentage,away_red_cards_minute_16_30,away_red_cards_minute_16_30_percentage,away_red_cards_minute_31_45,away_red_cards_minute_31_45_percentage,away_red_cards_minute_46_60,away_red_cards_minute_46_60_percentage,away_red_cards_minute_61_75,away_red_cards_minute_61_75_percentage,away_red_cards_minute_76_90,away_red_cards_minute_76_90_percentage,away_red_cards_minute_91_105,away_red_cards_minute_91_105_percentage,away_total_clean_sheets,away_total_goals_against,away_total_goals_for,away_yellow_cards_minute_0_15,away_yellow_cards_minute_0_15_percentage,away_yellow_cards_minute_16_30,away_yellow_cards_minute_16_30_percentage,away_yellow_cards_minute_31_45,away_yellow_cards_minute_31_45_percentage,away_yellow_cards_minute_46_60,away_yellow_cards_minute_46_60_percentage,away_yellow_cards_minute_61_75,away_yellow_cards_minute_61_75_percentage,away_yellow_cards_minute_76_90,away_yellow_cards_minute_76_90_percentage,away_yellow_cards_minute_91_105,away_yellow_cards_minute_91_105_percentage,home_average_away_goals_against,home_average_away_goals_for,home_average_home_goals_against,home_average_home_goals_for,home_average_total_goals_against,home_average_total_goals_for,home_away_clean_sheets,home_away_draws,home_away_goals_against,home_away_goals_for,home_away_loses,home_away_wins,home_biggest_away_goals_against,home_biggest_away_goals_for,home_biggest_away_lose,home_biggest_away_win,home_biggest_draw_streak,home_biggest_home_goals_against,home_biggest_home_goals_for,home_biggest_home_lose,home_biggest_home_win,home_biggest_lose_streak,home_biggest_win_streak,home_failed_to_score_away,home_failed_to_score_home,home_failed_to_score_total,home_form,home_goals_against_minute_0_15,home_goals_against_minute_0_15_percentage,home_goals_against_minute_16_30,home_goals_against_minute_16_30_percentage,home_goals_against_minute_31_45,home_

In [13]:
footballdata_wTeamStats.columns.to_list()

['AverageAwayWinOdd',
 'AverageDrawWinOdd',
 'AverageHomeWinOdd',
 'AvgOver2_5',
 'AvgUnder2_5',
 'AwayGoals',
 'AwayTeamId',
 'AwayTeamName',
 'Date',
 'Div',
 'FullTimeResult',
 'HomeGoals',
 'HomeTeamName',
 'HomeTeamid',
 'Season',
 'away_average_away_goals_against',
 'away_average_away_goals_for',
 'away_average_home_goals_against',
 'away_average_home_goals_for',
 'away_average_total_goals_against',
 'away_average_total_goals_for',
 'away_away_clean_sheets',
 'away_away_draws',
 'away_away_goals_against',
 'away_away_goals_for',
 'away_away_loses',
 'away_away_wins',
 'away_biggest_away_goals_against',
 'away_biggest_away_goals_for',
 'away_biggest_away_lose',
 'away_biggest_away_win',
 'away_biggest_draw_streak',
 'away_biggest_home_goals_against',
 'away_biggest_home_goals_for',
 'away_biggest_home_lose',
 'away_biggest_home_win',
 'away_biggest_lose_streak',
 'away_biggest_win_streak',
 'away_failed_to_score_away',
 'away_failed_to_score_home',
 'away_failed_to_score_total',
 

# Data Prep

## Remove Incomplete Columns & Non important (except indexes)

In [14]:
not_complete = ['away_red_cards_minute_0_15', 'away_red_cards_minute_0_15_percentage',
               'away_red_cards_minute_16_30', 'away_red_cards_minute_16_30_percentage',
               'away_red_cards_minute_31_45', 'away_red_cards_minute_31_45_percentage',
               'away_red_cards_minute_46_60', 'away_red_cards_minute_46_60_percentage',
               'away_red_cards_minute_61_75', 'away_red_cards_minute_61_75_percentage',
               'away_red_cards_minute_61_75', 'away_red_cards_minute_61_75_percentage',
               'away_red_cards_minute_76_90', 'away_red_cards_minute_76_90_percentage',
               'away_red_cards_minute_91_105', 'away_red_cards_minute_91_105_percentage',
               'away_yellow_cards_minute_0_15', 'away_yellow_cards_minute_0_15_percentage',
               'away_yellow_cards_minute_16_30', 'away_yellow_cards_minute_16_30_percentage',
               'away_yellow_cards_minute_31_45', 'away_yellow_cards_minute_31_45_percentage',
               'away_yellow_cards_minute_46_60', 'away_yellow_cards_minute_46_60_percentage',
               'away_yellow_cards_minute_61_75', 'away_yellow_cards_minute_61_75_percentage',
               'away_yellow_cards_minute_76_90', 'away_yellow_cards_minute_76_90_percentage',
               'away_yellow_cards_minute_91_105', 'away_yellow_cards_minute_91_105_percentage',
                'home_red_cards_minute_0_15', 'home_red_cards_minute_0_15_percentage',
               'home_red_cards_minute_16_30', 'home_red_cards_minute_16_30_percentage',
               'home_red_cards_minute_31_45', 'home_red_cards_minute_31_45_percentage',
               'home_red_cards_minute_46_60', 'home_red_cards_minute_46_60_percentage',
               'home_red_cards_minute_61_75', 'home_red_cards_minute_61_75_percentage',
               'home_red_cards_minute_61_75', 'home_red_cards_minute_61_75_percentage',
               'home_red_cards_minute_76_90', 'home_red_cards_minute_76_90_percentage',
               'home_red_cards_minute_91_105', 'home_red_cards_minute_91_105_percentage',
               'home_yellow_cards_minute_0_15', 'home_yellow_cards_minute_0_15_percentage',
               'home_yellow_cards_minute_16_30', 'home_yellow_cards_minute_16_30_percentage',
               'home_yellow_cards_minute_31_45', 'home_yellow_cards_minute_31_45_percentage',
               'home_yellow_cards_minute_46_60', 'home_yellow_cards_minute_46_60_percentage',
               'home_yellow_cards_minute_61_75', 'home_yellow_cards_minute_61_75_percentage',
               'home_yellow_cards_minute_76_90', 'home_yellow_cards_minute_76_90_percentage',
               'home_yellow_cards_minute_91_105', 'home_yellow_cards_minute_91_105_percentage']

# away average goals and home average goals will be removed because we will average all columns
not_features = [
               'away_biggest_away_lose', 'away_biggest_away_win', 'away_biggest_home_lose', 'away_biggest_home_win',
               'away_lineups',
               'home_biggest_away_lose', 'home_biggest_away_win', 'home_biggest_home_lose', 'home_biggest_home_win',
               'home_lineups',
    
               'away_biggest_away_goals_against', 'away_biggest_away_goals_for', 'away_biggest_draw_streak',
               'away_biggest_home_goals_against', 'away_biggest_home_goals_for', 'away_biggest_lose_streak',
               'away_biggest_win_streak',
               'home_biggest_away_goals_against', 'home_biggest_away_goals_for', 'home_biggest_draw_streak',
               'home_biggest_home_goals_against', 'home_biggest_home_goals_for', 'home_biggest_lose_streak',
               'home_biggest_win_streak',
    
               'away_average_away_goals_against', 'away_average_away_goals_for', 'away_average_home_goals_against',
               'away_average_home_goals_for', 'away_average_total_goals_against', 'away_average_total_goals_for',
               'home_average_away_goals_against', 'home_average_away_goals_for', 'home_average_home_goals_against',
               'home_average_home_goals_for', 'home_average_total_goals_against', 'home_average_total_goals_for',
    
               'away_penalties_missed', 'away_penalties_missed_percentage', 'away_penalties_scored',
               'away_penalties_scored', 'away_penalties_scored_percentage',
               'home_penalties_missed', 'home_penalties_missed_percentage', 'home_penalties_scored',
               'home_penalties_scored', 'home_penalties_scored_percentage'
               ]

In [15]:
footballdata_wTeamStats = \
    footballdata_wTeamStats[list(set(footballdata_wTeamStats.columns.tolist()) - set(not_complete + not_features))]
footballdata_wTeamStats

,home_goals_against_minute_61_75,away_home_goals_for,away_goals_against_minute_76_90_percentage,home_goals_for_minute_76_90,away_goals_for_minute_31_45_percentage,AwayGoals,away_goals_against_minute_31_45,home_total_clean_sheets,away_failed_to_score_home,home_goals_for_minute_31_45_percentage,away_goals_against_minute_16_30,home_goals_for_minute_16_30,home_goals_for_minute_91_105_percentage,away_failed_to_score_total,away_goals_for_minute_31_45,away_home_draws,home_away_draws,home_goals_against_minute_0_15,away_goals_for_minute_76_90,away_goals_for_minute_76_90_percentage,away_away_goals_against,home_form,away_goals_for_minute_0_15,home_goals_against_minute_46_60,HomeTeamName,home_away_goals_against,away_goals_for_minute_16_30_percentage,away_away_wins,away_penalties_for,home_goals_for_minute_61_75_percentage,away_home_clean_sheets,home_goals_for_minute_16_30_percentage,away_goals_against_minute_0_15_percentage,home_failed_to_score_home,away_goals_against_minute_31_45_percentage,away_goals_against_minute_16_30_percentage,away_goals_against_minute_76_90,home_failed_to_score_away,away_goals_against_minute_61_75_percentage,away_goals_for_minute_0_15_percentage,away_away_goals_for,away_goals_against_minute_91_105_percentage,home_goals_against_minute_31_45_percentage,away_home_wins,away_goals_for_minute_91_105_percentage,away_goals_against_minute_61_75,Date,away_total_clean_sheets,home_goals_for_minute_0_15,away_goals_for_minute_16_30,home_away_wins,away_form,away_total_goals_against,away_away_clean_sheets,home_goals_for_minute_31_45,AvgUnder2_5,home_away_goals_for,home_goals_against_minute_76_90,away_goals_for_minute_46_60_percentage,home_home_loses,home_goals_for_minute_0_15_percentage,Season,home_home_clean_sheets,home_failed_to_score_total,home_penalties_for,AwayTeamId,AverageDrawWinOdd,AwayTeamName,AverageHomeWinOdd,home_away_loses,Div,away_goals_against_minute_91_105,away_goals_against_minute_46_60,home_away_clean_sheets,home_goals_for_minute_76_90_percentage,AverageAwayWinOdd,home_total_goals_for,home_home_goals_for,away_goals_against_minute_46_60_percentage,home_home_wins,away_goals_against_minute_0_15,home_goals_against_minute_16_30,home_goals_against_minute_91_105_percentage,away_total_goals_for,away_home_loses,home_goals_for_minute_61_75,AvgOver2_5,away_goals_for_minute_46_60,away_goals_for_minute_61_75,home_home_draws,home_goals_against_minute_46_60_percentage,FullTimeResult,HomeGoals,home_goals_against_minute_61_75_percentage,home_total_goals_against,away_home_goals_against,home_goals_for_minute_46_60,home_goals_for_minute_46_60_percentage,away_away_loses,home_goals_against_minute_16_30_percentage,away_goals_for_minute_91_105,home_home_goals_against,HomeTeamid,home_goals_for_minute_91_105,home_goals_against_minute_91_105,away_away_draws,away_goals_for_minute_61_75_percentage,home_goals_against_minute_76_90_percentage,away_failed_to_score_away,home_goals_against_minute_31_45,home_goals_against_minute_0_15_percentage
0,NaN,0.0,NaN,NaN,NaN,1.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,Standard,0.0,NaN,0.0,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,30/07/2010,0.0,NaN,NaN,0.0,NaN,0.0,0.0,NaN,1.83,0.0,NaN,NaN,0.0,NaN,2010.0,0.0,0.0,0.0,600.0,3.60,Waregem,1.73,0.0,144.0,NaN,NaN,0.0,NaN,4.75,0.0,0.0,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN,1.93,NaN,NaN,0.0,NaN,D,1.0,NaN,0.0,0.0,NaN,NaN,0.0,NaN,NaN,0.0,733.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN
1,NaN,0.0,NaN,NaN,NaN,1.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,Anderlecht,0.0,NaN,0.0,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,31/07/2010,0.0,NaN,NaN,0.0,NaN,0.0,0.0,NaN,2.41,0.0,NaN,NaN,0.0,NaN,2010.0,0.0,0.0,0.0,739.0,6.50,Eupen,1.20,0.0,144.0,NaN,NaN,0.0,NaN,10.00,0.0,0.0,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN,1.51,NaN,NaN,0.0,NaN,H,4.0,NaN,0.0,0.0,NaN,NaN,0.0,NaN,NaN,0.0,554.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN
2,NaN,0.0,NaN,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,NaN,

## Convert NaN/Nulls to 0

In [21]:
nan_to_zero = ['away_goals_against_minute_0_15', 'away_goals_against_minute_0_15_percentage',
            'away_goals_against_minute_16_30', 'away_goals_against_minute_16_30_percentage',
            'away_goals_against_minute_16_30_percentage', 'away_goals_against_minute_31_45',
             'away_goals_against_minute_31_45_percentage', 'away_goals_against_minute_46_60',
            'away_goals_against_minute_46_60_percentage', 'away_goals_against_minute_61_75',
            'away_goals_against_minute_61_75_percentage', 'away_goals_against_minute_76_90',
            'away_goals_against_minute_76_90_percentage', 'away_goals_against_minute_91_105',
            'away_goals_against_minute_91_105_percentage', 'away_goals_for_minute_0_15',
            'away_goals_for_minute_0_15_percentage', 'away_goals_for_minute_16_30', 'away_goals_for_minute_16_30_percentage',
            'away_goals_for_minute_31_45', 'away_goals_for_minute_31_45_percentage', 'away_goals_for_minute_46_60',
            'away_goals_for_minute_46_60_percentage', 'away_goals_for_minute_61_75', 'away_goals_for_minute_61_75_percentage',
            'away_goals_for_minute_76_90', 'away_goals_for_minute_76_90_percentage', 'away_goals_for_minute_91_105',
            'away_goals_for_minute_91_105_percentage',
             'home_goals_against_minute_0_15', 'home_goals_against_minute_0_15_percentage',
            'home_goals_against_minute_16_30', 'home_goals_against_minute_16_30_percentage',
            'home_goals_against_minute_16_30_percentage', 'home_goals_against_minute_31_45',
             'home_goals_against_minute_31_45_percentage', 'home_goals_against_minute_46_60',
            'home_goals_against_minute_46_60_percentage', 'home_goals_against_minute_61_75',
            'home_goals_against_minute_61_75_percentage', 'home_goals_against_minute_76_90',
            'home_goals_against_minute_76_90_percentage', 'home_goals_against_minute_91_105',
            'home_goals_against_minute_91_105_percentage', 'home_goals_for_minute_0_15',
            'home_goals_for_minute_0_15_percentage', 'home_goals_for_minute_16_30', 'home_goals_for_minute_16_30_percentage',
            'home_goals_for_minute_31_45', 'home_goals_for_minute_31_45_percentage', 'home_goals_for_minute_46_60',
            'home_goals_for_minute_46_60_percentage', 'home_goals_for_minute_61_75', 'home_goals_for_minute_61_75_percentage',
            'home_goals_for_minute_76_90', 'home_goals_for_minute_76_90_percentage', 'home_goals_for_minute_91_105',
            'home_goals_for_minute_91_105_percentage']

In [22]:
print(footballdata_wTeamStats.count().sort_values().to_string())

home_form                                      6210
away_form                                      6217
home_goals_against_minute_61_75                6654
away_goals_against_minute_0_15                 6654
home_home_wins                                 6654
away_goals_against_minute_46_60_percentage     6654
home_home_goals_for                            6654
home_total_goals_for                           6654
AverageAwayWinOdd                              6654
home_goals_for_minute_76_90_percentage         6654
home_away_clean_sheets                         6654
away_goals_against_minute_46_60                6654
away_goals_against_minute_91_105               6654
Div                                            6654
home_away_loses                                6654
AverageHomeWinOdd                              6654
AwayTeamName                                   6654
AverageDrawWinOdd                              6654
AwayTeamId                                     6654
home_penalti

In [18]:
for column in footballdata_wTeamStats.columns:
    if 'percentage' in column:
        footballdata_wTeamStats[column] = footballdata_wTeamStats[column].str.slice(0,-1,1).astype(float)

for column in nan_to_zero:
    if 'percentage' in column:
        footballdata_wTeamStats[column] = footballdata_wTeamStats[column].fillna(0.0)
    else:
        footballdata_wTeamStats[column] = footballdata_wTeamStats[column].fillna(0).astype(int)
footballdata_wTeamStats

<ipython-input-18-04bdf39b3b3a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  footballdata_wTeamStats[column] = footballdata_wTeamStats[column].str.slice(0,-1,1).astype(float)
<ipython-input-18-04bdf39b3b3a>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  footballdata_wTeamStats[column] = footballdata_wTeamStats[column].fillna(0).astype(int)
<ipython-input-18-04bdf39b3b3a>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

,home_goals_against_minute_61_75,away_home_goals_for,away_goals_against_minute_76_90_percentage,home_goals_for_minute_76_90,away_goals_for_minute_31_45_percentage,AwayGoals,away_goals_against_minute_31_45,home_total_clean_sheets,away_failed_to_score_home,home_goals_for_minute_31_45_percentage,away_goals_against_minute_16_30,home_goals_for_minute_16_30,home_goals_for_minute_91_105_percentage,away_failed_to_score_total,away_goals_for_minute_31_45,away_home_draws,home_away_draws,home_goals_against_minute_0_15,away_goals_for_minute_76_90,away_goals_for_minute_76_90_percentage,away_away_goals_against,home_form,away_goals_for_minute_0_15,home_goals_against_minute_46_60,HomeTeamName,home_away_goals_against,away_goals_for_minute_16_30_percentage,away_away_wins,away_penalties_for,home_goals_for_minute_61_75_percentage,away_home_clean_sheets,home_goals_for_minute_16_30_percentage,away_goals_against_minute_0_15_percentage,home_failed_to_score_home,away_goals_against_minute_31_45_percentage,away_goals_against_minute_16_30_percentage,away_goals_against_minute_76_90,home_failed_to_score_away,away_goals_against_minute_61_75_percentage,away_goals_for_minute_0_15_percentage,away_away_goals_for,away_goals_against_minute_91_105_percentage,home_goals_against_minute_31_45_percentage,away_home_wins,away_goals_for_minute_91_105_percentage,away_goals_against_minute_61_75,Date,away_total_clean_sheets,home_goals_for_minute_0_15,away_goals_for_minute_16_30,home_away_wins,away_form,away_total_goals_against,away_away_clean_sheets,home_goals_for_minute_31_45,AvgUnder2_5,home_away_goals_for,home_goals_against_minute_76_90,away_goals_for_minute_46_60_percentage,home_home_loses,home_goals_for_minute_0_15_percentage,Season,home_home_clean_sheets,home_failed_to_score_total,home_penalties_for,AwayTeamId,AverageDrawWinOdd,AwayTeamName,AverageHomeWinOdd,home_away_loses,Div,away_goals_against_minute_91_105,away_goals_against_minute_46_60,home_away_clean_sheets,home_goals_for_minute_76_90_percentage,AverageAwayWinOdd,home_total_goals_for,home_home_goals_for,away_goals_against_minute_46_60_percentage,home_home_wins,away_goals_against_minute_0_15,home_goals_against_minute_16_30,home_goals_against_minute_91_105_percentage,away_total_goals_for,away_home_loses,home_goals_for_minute_61_75,AvgOver2_5,away_goals_for_minute_46_60,away_goals_for_minute_61_75,home_home_draws,home_goals_against_minute_46_60_percentage,FullTimeResult,HomeGoals,home_goals_against_minute_61_75_percentage,home_total_goals_against,away_home_goals_against,home_goals_for_minute_46_60,home_goals_for_minute_46_60_percentage,away_away_loses,home_goals_against_minute_16_30_percentage,away_goals_for_minute_91_105,home_home_goals_against,HomeTeamid,home_goals_for_minute_91_105,home_goals_against_minute_91_105,away_away_draws,away_goals_for_minute_61_75_percentage,home_goals_against_minute_76_90_percentage,away_failed_to_score_away,home_goals_against_minute_31_45,home_goals_against_minute_0_15_percentage
0,0,0.0,0.0,0,0.0,1.0,0,0.0,0.0,0.0,0,0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0,NaN,0,0,Standard,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0,30/07/2010,0.0,0,0,0.0,NaN,0.0,0.0,0,1.83,0.0,0,0.0,0.0,0.0,2010.0,0.0,0.0,0.0,600.0,3.60,Waregem,1.73,0.0,144.0,0,0,0.0,0.0,4.75,0.0,0.0,0.00,0.0,0,0,0.0,0.0,0.0,0,1.93,0,0,0.0,0.0,D,1.0,0.0,0.0,0.0,0,0.00,0.0,0.0,0,0.0,733.0,0,0,0.0,0.0,0.0,0.0,0,0.0
1,0,0.0,0.0,0,0.0,1.0,0,0.0,0.0,0.0,0,0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0,NaN,0,0,Anderlecht,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0,31/07/2010,0.0,0,0,0.0,NaN,0.0,0.0,0,2.41,0.0,0,0.0,0.0,0.0,2010.0,0.0,0.0,0.0,739.0,6.50,Eupen,1.20,0.0,144.0,0,0,0.0,0.0,10.00,0.0,0.0,0.00,0.0,0,0,0.0,0.0,0.0,0,1.51,0,0,0.0,0.0,H,4.0,0.0,0.0,0.0,0,0.00,0.0,0.0,0,0.0,554.0,0,0,0.0,0.0,0.0,0.0,0,0.0
2,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0,0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0,NaN,0,0,Kortrijk,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0,31/07/201

In [19]:
footballdata_wTeamStats.tail(5)

,home_goals_against_minute_61_75,away_home_goals_for,away_goals_against_minute_76_90_percentage,home_goals_for_minute_76_90,away_goals_for_minute_31_45_percentage,AwayGoals,away_goals_against_minute_31_45,home_total_clean_sheets,away_failed_to_score_home,home_goals_for_minute_31_45_percentage,away_goals_against_minute_16_30,home_goals_for_minute_16_30,home_goals_for_minute_91_105_percentage,away_failed_to_score_total,away_goals_for_minute_31_45,away_home_draws,home_away_draws,home_goals_against_minute_0_15,away_goals_for_minute_76_90,away_goals_for_minute_76_90_percentage,away_away_goals_against,home_form,away_goals_for_minute_0_15,home_goals_against_minute_46_60,HomeTeamName,home_away_goals_against,away_goals_for_minute_16_30_percentage,away_away_wins,away_penalties_for,home_goals_for_minute_61_75_percentage,away_home_clean_sheets,home_goals_for_minute_16_30_percentage,away_goals_against_minute_0_15_percentage,home_failed_to_score_home,away_goals_against_minute_31_45_percentage,away_goals_against_minute_16_30_percentage,away_goals_against_minute_76_90,home_failed_to_score_away,away_goals_against_minute_61_75_percentage,away_goals_for_minute_0_15_percentage,away_away_goals_for,away_goals_against_minute_91_105_percentage,home_goals_against_minute_31_45_percentage,away_home_wins,away_goals_for_minute_91_105_percentage,away_goals_against_minute_61_75,Date,away_total_clean_sheets,home_goals_for_minute_0_15,away_goals_for_minute_16_30,home_away_wins,away_form,away_total_goals_against,away_away_clean_sheets,home_goals_for_minute_31_45,AvgUnder2_5,home_away_goals_for,home_goals_against_minute_76_90,away_goals_for_minute_46_60_percentage,home_home_loses,home_goals_for_minute_0_15_percentage,Season,home_home_clean_sheets,home_failed_to_score_total,home_penalties_for,AwayTeamId,AverageDrawWinOdd,AwayTeamName,AverageHomeWinOdd,home_away_loses,Div,away_goals_against_minute_91_105,away_goals_against_minute_46_60,home_away_clean_sheets,home_goals_for_minute_76_90_percentage,AverageAwayWinOdd,home_total_goals_for,home_home_goals_for,away_goals_against_minute_46_60_percentage,home_home_wins,away_goals_against_minute_0_15,home_goals_against_minute_16_30,home_goals_against_minute_91_105_percentage,away_total_goals_for,away_home_loses,home_goals_for_minute_61_75,AvgOver2_5,away_goals_for_minute_46_60,away_goals_for_minute_61_75,home_home_draws,home_goals_against_minute_46_60_percentage,FullTimeResult,HomeGoals,home_goals_against_minute_61_75_percentage,home_total_goals_against,away_home_goals_against,home_goals_for_minute_46_60,home_goals_for_minute_46_60_percentage,away_away_loses,home_goals_against_minute_16_30_percentage,away_goals_for_minute_91_105,home_home_goals_against,HomeTeamid,home_goals_for_minute_91_105,home_goals_against_minute_91_105,away_away_draws,away_goals_for_minute_61_75_percentage,home_goals_against_minute_76_90_percentage,away_failed_to_score_away,home_goals_against_minute_31_45,home_goals_against_minute_0_15_percentage
3602,0,0.0,0.0,0,0.0,0.0,0,0.0,1.0,40.0,0,0,0.0,1.0,0,1.0,0.0,0,0,0.0,0.0,W,0,1,Feyenoord,2.0,0.0,0.0,0.0,60.00,1.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0,13/08/2022,1.0,0,0,1.0,D,0.0,0.0,2,2.67,5.0,0,0.0,0.0,0.0,2022.0,0.0,0.0,0.0,210.0,5.79,Heerenveen,1.29,0.0,88.0,0,0,0.0,0.0,8.97,5.0,0.0,0.00,0.0,0,1,0.0,0.0,0.0,3,1.45,0,0,0.0,50.0,D,0.0,0.0,2.0,0.0,0,0.00,0.0,50.0,0,0.0,209.0,0,0,0.0,0.0,0.0,0.0,0,0.0
3603,0,0.0,0.0,0,0.0,4.0,0,0.0,1.0,50.0,0,0,0.0,1.0,0,0.0,1.0,1,0,0.0,0.0,D,0,0,Volendam,2.0,0.0,0.0,0.0,50.00,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.00,0.0,0.0,100.0,50.0,0.0,0.0,0,14/08/2022,0.0,0,0,0.0,L,1.0,0.0,1,2.24,2.0,0,0.0,0.0,0.0,2022.0,0.0,0.0,0.0,413.0,3.63,Nijmegen,2.55,0.0,88.0,1,0,0.0,0.0,2.56,2.0,0.0,0.00,0.0,0,0,0.0,0.0,1.0,1,1.64,0,0,0.0,0.0,A,1.0,0.0,2.0,1.0,0,0.00,0.0,0.0,0,0.0,416.0,0,0,0.0,0.0,0.0,0.0,1,50.0
3604,0,2.0,0.0,0,50.0,1.0,1,0.0,0.0,0.0,0,0,0.0,0.0,1,1.0,0.0,1,0,0.0,0.0,W,1,0,Ajax,2.0,0.0,0.0,0.0,33.33,0.0,0.0,0.0,0.0,50.0,0.0,0,0.0,50.00,50.0,0.0,0.0,0.0,0.0,0.0

In [20]:
print(footballdata_wTeamStats.count().sort_values().to_string())

home_form                                      6210
away_form                                      6217
home_goals_against_minute_61_75                6654
away_goals_against_minute_0_15                 6654
home_home_wins                                 6654
away_goals_against_minute_46_60_percentage     6654
home_home_goals_for                            6654
home_total_goals_for                           6654
AverageAwayWinOdd                              6654
home_goals_for_minute_76_90_percentage         6654
home_away_clean_sheets                         6654
away_goals_against_minute_46_60                6654
away_goals_against_minute_91_105               6654
Div                                            6654
home_away_loses                                6654
AverageHomeWinOdd                              6654
AwayTeamName                                   6654
AverageDrawWinOdd                              6654
AwayTeamId                                     6654
home_penalti

## Modifications

### Add Total wins, draws & losses

In [23]:
footballdata_wTeamStats['home_total_wins'] = \
    footballdata_wTeamStats['home_home_wins'] + footballdata_wTeamStats['home_away_wins']
footballdata_wTeamStats['home_total_draws'] = \
    footballdata_wTeamStats['home_home_draws'] + footballdata_wTeamStats['home_away_draws']
footballdata_wTeamStats['home_total_loses'] = \
    footballdata_wTeamStats['home_home_loses'] + footballdata_wTeamStats['home_away_loses']

# TODO: total games is computed bases only on the home games (might require future change)
footballdata_wTeamStats['home_total_games'] = footballdata_wTeamStats['home_total_wins']+\
    footballdata_wTeamStats['home_total_draws']+footballdata_wTeamStats['home_total_loses']

footballdata_wTeamStats['away_total_wins'] = \
    footballdata_wTeamStats['away_home_wins'] + footballdata_wTeamStats['away_away_wins']
footballdata_wTeamStats['away_total_draws'] = \
    footballdata_wTeamStats['away_home_draws'] + footballdata_wTeamStats['away_away_draws']
footballdata_wTeamStats['away_total_loses'] = \
    footballdata_wTeamStats['away_home_loses'] + footballdata_wTeamStats['away_away_loses']

footballdata_wTeamStats['away_total_games'] = footballdata_wTeamStats['away_total_wins']+\
    footballdata_wTeamStats['away_total_draws']+footballdata_wTeamStats['away_total_loses']

footballdata_wTeamStats['home_total_home_games'] = footballdata_wTeamStats['home_home_wins']+\
    footballdata_wTeamStats['home_home_draws']+footballdata_wTeamStats['home_home_loses']
footballdata_wTeamStats['home_total_away_games'] = footballdata_wTeamStats['home_away_wins']+\
    footballdata_wTeamStats['home_away_draws']+footballdata_wTeamStats['home_away_loses']

footballdata_wTeamStats['away_total_home_games'] = footballdata_wTeamStats['away_home_wins']+\
    footballdata_wTeamStats['away_home_draws']+footballdata_wTeamStats['away_home_loses']
footballdata_wTeamStats['away_total_away_games'] = footballdata_wTeamStats['away_away_wins']+\
    footballdata_wTeamStats['away_away_draws']+footballdata_wTeamStats['away_away_loses']

footballdata_wTeamStats

<ipython-input-23-26fe08bd73ab>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  footballdata_wTeamStats['home_total_wins'] = \
<ipython-input-23-26fe08bd73ab>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  footballdata_wTeamStats['home_total_draws'] = \
<ipython-input-23-26fe08bd73ab>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

,home_goals_against_minute_61_75,away_home_goals_for,away_goals_against_minute_76_90_percentage,home_goals_for_minute_76_90,away_goals_for_minute_31_45_percentage,AwayGoals,away_goals_against_minute_31_45,home_total_clean_sheets,away_failed_to_score_home,home_goals_for_minute_31_45_percentage,away_goals_against_minute_16_30,home_goals_for_minute_16_30,home_goals_for_minute_91_105_percentage,away_failed_to_score_total,away_goals_for_minute_31_45,away_home_draws,home_away_draws,home_goals_against_minute_0_15,away_goals_for_minute_76_90,away_goals_for_minute_76_90_percentage,away_away_goals_against,home_form,away_goals_for_minute_0_15,home_goals_against_minute_46_60,HomeTeamName,home_away_goals_against,away_goals_for_minute_16_30_percentage,away_away_wins,away_penalties_for,home_goals_for_minute_61_75_percentage,away_home_clean_sheets,home_goals_for_minute_16_30_percentage,away_goals_against_minute_0_15_percentage,home_failed_to_score_home,away_goals_against_minute_31_45_percentage,away_goals_against_minute_16_30_percentage,away_goals_against_minute_76_90,home_failed_to_score_away,away_goals_against_minute_61_75_percentage,away_goals_for_minute_0_15_percentage,away_away_goals_for,away_goals_against_minute_91_105_percentage,home_goals_against_minute_31_45_percentage,away_home_wins,away_goals_for_minute_91_105_percentage,away_goals_against_minute_61_75,Date,away_total_clean_sheets,home_goals_for_minute_0_15,away_goals_for_minute_16_30,home_away_wins,away_form,away_total_goals_against,away_away_clean_sheets,home_goals_for_minute_31_45,AvgUnder2_5,home_away_goals_for,home_goals_against_minute_76_90,away_goals_for_minute_46_60_percentage,home_home_loses,home_goals_for_minute_0_15_percentage,Season,home_home_clean_sheets,home_failed_to_score_total,home_penalties_for,AwayTeamId,AverageDrawWinOdd,AwayTeamName,AverageHomeWinOdd,home_away_loses,Div,away_goals_against_minute_91_105,away_goals_against_minute_46_60,home_away_clean_sheets,home_goals_for_minute_76_90_percentage,AverageAwayWinOdd,home_total_goals_for,home_home_goals_for,away_goals_against_minute_46_60_percentage,home_home_wins,away_goals_against_minute_0_15,home_goals_against_minute_16_30,home_goals_against_minute_91_105_percentage,away_total_goals_for,away_home_loses,home_goals_for_minute_61_75,AvgOver2_5,away_goals_for_minute_46_60,away_goals_for_minute_61_75,home_home_draws,home_goals_against_minute_46_60_percentage,FullTimeResult,HomeGoals,home_goals_against_minute_61_75_percentage,home_total_goals_against,away_home_goals_against,home_goals_for_minute_46_60,home_goals_for_minute_46_60_percentage,away_away_loses,home_goals_against_minute_16_30_percentage,away_goals_for_minute_91_105,home_home_goals_against,HomeTeamid,home_goals_for_minute_91_105,home_goals_against_minute_91_105,away_away_draws,away_goals_for_minute_61_75_percentage,home_goals_against_minute_76_90_percentage,away_failed_to_score_away,home_goals_against_minute_31_45,home_goals_against_minute_0_15_percentage,home_total_wins,home_total_draws,home_total_loses,home_total_games,away_total_wins,away_total_draws,away_total_loses,away_total_games,home_total_home_games,home_total_away_games,away_total_home_games,away_total_away_games
0,0,0.0,0.0,0,0.0,1.0,0,0.0,0.0,0.0,0,0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0,NaN,0,0,Standard,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0,30/07/2010,0.0,0,0,0.0,NaN,0.0,0.0,0,1.83,0.0,0,0.0,0.0,0.0,2010.0,0.0,0.0,0.0,600.0,3.60,Waregem,1.73,0.0,144.0,0,0,0.0,0.0,4.75,0.0,0.0,0.00,0.0,0,0,0.0,0.0,0.0,0,1.93,0,0,0.0,0.0,D,1.0,0.0,0.0,0.0,0,0.00,0.0,0.0,0,0.0,733.0,0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0.0,0.0,0,0.0,1.0,0,0.0,0.0,0.0,0,0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0,NaN,0,0,Anderlecht,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0,31/07/2010,0.0,0,0,0.0,NaN,0.0,0.0,0,2.41,0.0,0,0.0,0.0,0.0,2010.0,0.0,0.0,0.0,739.0,6.50,Eupen,1.20,0.0,144.0,0,0,0.0,0.0,10.00,0.0,0.0,0.00,0.0,0,0,0.0,0.0,0.

### Away Team & Home Team Form

In [24]:
to_modify = ['away_form', 'home_form']

In [25]:
# Same as before but normalized
games = [1, 2, 5, 10]
for column in to_modify:
    footballdata_wTeamStats[column] = footballdata_wTeamStats[column].fillna('')
    for nr in games:
        footballdata_wTeamStats[column+f"_last{nr}games_wins"] = \
            footballdata_wTeamStats[column].apply(lambda x : x[-nr:]).str.count('W') / \
            footballdata_wTeamStats['home_total_games'].clip(upper=10)
        footballdata_wTeamStats[column+f"_last{nr}games_draws"] = \
            footballdata_wTeamStats[column].apply(lambda x : x[-nr:]).str.count('D') / \
            footballdata_wTeamStats['home_total_games'].clip(upper=10)
        footballdata_wTeamStats[column+f"_last{nr}games_loses"] = \
            footballdata_wTeamStats[column].apply(lambda x : x[-nr:]).str.count('L') / \
            footballdata_wTeamStats['home_total_games'].clip(upper=10)
footballdata_wTeamStats

<ipython-input-25-4b1632eb5d92>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  footballdata_wTeamStats[column] = footballdata_wTeamStats[column].fillna('')
<ipython-input-25-4b1632eb5d92>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  footballdata_wTeamStats[column+f"_last{nr}games_wins"] = \
<ipython-input-25-4b1632eb5d92>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

,home_goals_against_minute_61_75,away_home_goals_for,away_goals_against_minute_76_90_percentage,home_goals_for_minute_76_90,away_goals_for_minute_31_45_percentage,AwayGoals,away_goals_against_minute_31_45,home_total_clean_sheets,away_failed_to_score_home,home_goals_for_minute_31_45_percentage,away_goals_against_minute_16_30,home_goals_for_minute_16_30,home_goals_for_minute_91_105_percentage,away_failed_to_score_total,away_goals_for_minute_31_45,away_home_draws,home_away_draws,home_goals_against_minute_0_15,away_goals_for_minute_76_90,away_goals_for_minute_76_90_percentage,away_away_goals_against,home_form,away_goals_for_minute_0_15,home_goals_against_minute_46_60,HomeTeamName,home_away_goals_against,away_goals_for_minute_16_30_percentage,away_away_wins,away_penalties_for,home_goals_for_minute_61_75_percentage,away_home_clean_sheets,home_goals_for_minute_16_30_percentage,away_goals_against_minute_0_15_percentage,home_failed_to_score_home,away_goals_against_minute_31_45_percentage,away_goals_against_minute_16_30_percentage,away_goals_against_minute_76_90,home_failed_to_score_away,away_goals_against_minute_61_75_percentage,away_goals_for_minute_0_15_percentage,away_away_goals_for,away_goals_against_minute_91_105_percentage,home_goals_against_minute_31_45_percentage,away_home_wins,away_goals_for_minute_91_105_percentage,away_goals_against_minute_61_75,Date,away_total_clean_sheets,home_goals_for_minute_0_15,away_goals_for_minute_16_30,home_away_wins,away_form,away_total_goals_against,away_away_clean_sheets,home_goals_for_minute_31_45,AvgUnder2_5,home_away_goals_for,home_goals_against_minute_76_90,away_goals_for_minute_46_60_percentage,home_home_loses,home_goals_for_minute_0_15_percentage,Season,home_home_clean_sheets,home_failed_to_score_total,home_penalties_for,AwayTeamId,AverageDrawWinOdd,AwayTeamName,AverageHomeWinOdd,home_away_loses,Div,away_goals_against_minute_91_105,away_goals_against_minute_46_60,home_away_clean_sheets,home_goals_for_minute_76_90_percentage,AverageAwayWinOdd,home_total_goals_for,home_home_goals_for,away_goals_against_minute_46_60_percentage,home_home_wins,away_goals_against_minute_0_15,home_goals_against_minute_16_30,home_goals_against_minute_91_105_percentage,away_total_goals_for,away_home_loses,home_goals_for_minute_61_75,AvgOver2_5,away_goals_for_minute_46_60,away_goals_for_minute_61_75,home_home_draws,home_goals_against_minute_46_60_percentage,FullTimeResult,HomeGoals,home_goals_against_minute_61_75_percentage,home_total_goals_against,away_home_goals_against,home_goals_for_minute_46_60,home_goals_for_minute_46_60_percentage,away_away_loses,home_goals_against_minute_16_30_percentage,away_goals_for_minute_91_105,home_home_goals_against,HomeTeamid,home_goals_for_minute_91_105,home_goals_against_minute_91_105,away_away_draws,away_goals_for_minute_61_75_percentage,home_goals_against_minute_76_90_percentage,away_failed_to_score_away,home_goals_against_minute_31_45,home_goals_against_minute_0_15_percentage,home_total_wins,home_total_draws,home_total_loses,home_total_games,away_total_wins,away_total_draws,away_total_loses,away_total_games,home_total_home_games,home_total_away_games,away_total_home_games,away_total_away_games,away_form_last1games_wins,away_form_last1games_draws,away_form_last1games_loses,away_form_last2games_wins,away_form_last2games_draws,away_form_last2games_loses,away_form_last5games_wins,away_form_last5games_draws,away_form_last5games_loses,away_form_last10games_wins,away_form_last10games_draws,away_form_last10games_loses,home_form_last1games_wins,home_form_last1games_draws,home_form_last1games_loses,home_form_last2games_wins,home_form_last2games_draws,home_form_last2games_loses,home_form_last5games_wins,home_form_last5games_draws,home_form_last5games_loses,home_form_last10games_wins,home_form_last10games_draws,home_form_last10games_loses
0,0,0.0,0.0,0,0.0,1.0,0,0.0,0.0,0.0,0,0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0,,0,0,Standard,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.00,0.0,0.0,0.0,0.0

In [26]:
footballdata_wTeamStats = footballdata_wTeamStats.drop(columns=to_modify)

In [27]:
print(footballdata_wTeamStats.dtypes.sort_values().to_string())

home_goals_against_minute_61_75                  int32
home_goals_for_minute_0_15                       int32
away_goals_for_minute_16_30                      int32
home_goals_against_minute_31_45                  int32
home_goals_against_minute_16_30                  int32
home_goals_for_minute_31_45                      int32
away_goals_against_minute_76_90                  int32
home_goals_against_minute_76_90                  int32
home_goals_against_minute_91_105                 int32
home_goals_for_minute_91_105                     int32
away_goals_for_minute_91_105                     int32
home_goals_for_minute_46_60                      int32
away_goals_against_minute_61_75                  int32
away_goals_for_minute_0_15                       int32
away_goals_for_minute_76_90                      int32
home_goals_against_minute_0_15                   int32
home_goals_against_minute_46_60                  int32
away_goals_against_minute_91_105                 int32
away_goals

### Remove 1st 5 "jornadas"

In [28]:
footballdata_wTeamStats = footballdata_wTeamStats[footballdata_wTeamStats['home_total_games']>=5]
footballdata_wTeamStats.reindex(sorted(footballdata_wTeamStats.columns), axis=1)

,AverageAwayWinOdd,AverageDrawWinOdd,AverageHomeWinOdd,AvgOver2_5,AvgUnder2_5,AwayGoals,AwayTeamId,AwayTeamName,Date,Div,FullTimeResult,HomeGoals,HomeTeamName,HomeTeamid,Season,away_away_clean_sheets,away_away_draws,away_away_goals_against,away_away_goals_for,away_away_loses,away_away_wins,away_failed_to_score_away,away_failed_to_score_home,away_failed_to_score_total,away_form_last10games_draws,away_form_last10games_loses,away_form_last10games_wins,away_form_last1games_draws,away_form_last1games_loses,away_form_last1games_wins,away_form_last2games_draws,away_form_last2games_loses,away_form_last2games_wins,away_form_last5games_draws,away_form_last5games_loses,away_form_last5games_wins,away_goals_against_minute_0_15,away_goals_against_minute_0_15_percentage,away_goals_against_minute_16_30,away_goals_against_minute_16_30_percentage,away_goals_against_minute_31_45,away_goals_against_minute_31_45_percentage,away_goals_against_minute_46_60,away_goals_against_minute_46_60_percentage,away_goals_against_minute_61_75,away_goals_against_minute_61_75_percentage,away_goals_against_minute_76_90,away_goals_against_minute_76_90_percentage,away_goals_against_minute_91_105,away_goals_against_minute_91_105_percentage,away_goals_for_minute_0_15,away_goals_for_minute_0_15_percentage,away_goals_for_minute_16_30,away_goals_for_minute_16_30_percentage,away_goals_for_minute_31_45,away_goals_for_minute_31_45_percentage,away_goals_for_minute_46_60,away_goals_for_minute_46_60_percentage,away_goals_for_minute_61_75,away_goals_for_minute_61_75_percentage,away_goals_for_minute_76_90,away_goals_for_minute_76_90_percentage,away_goals_for_minute_91_105,away_goals_for_minute_91_105_percentage,away_home_clean_sheets,away_home_draws,away_home_goals_against,away_home_goals_for,away_home_loses,away_home_wins,away_penalties_for,away_total_away_games,away_total_clean_sheets,away_total_draws,away_total_games,away_total_goals_against,away_total_goals_for,away_total_home_games,away_total_loses,away_total_wins,home_away_clean_sheets,home_away_draws,home_away_goals_against,home_away_goals_for,home_away_loses,home_away_wins,home_failed_to_score_away,home_failed_to_score_home,home_failed_to_score_total,home_form_last10games_draws,home_form_last10games_loses,home_form_last10games_wins,home_form_last1games_draws,home_form_last1games_loses,home_form_last1games_wins,home_form_last2games_draws,home_form_last2games_loses,home_form_last2games_wins,home_form_last5games_draws,home_form_last5games_loses,home_form_last5games_wins,home_goals_against_minute_0_15,home_goals_against_minute_0_15_percentage,home_goals_against_minute_16_30,home_goals_against_minute_16_30_percentage,home_goals_against_minute_31_45,home_goals_against_minute_31_45_percentage,home_goals_against_minute_46_60,home_goals_against_minute_46_60_percentage,home_goals_against_minute_61_75,home_goals_against_minute_61_75_percentage,home_goals_against_minute_76_90,home_goals_against_minute_76_90_percentage,home_goals_against_minute_91_105,home_goals_against_minute_91_105_percentage,home_goals_for_minute_0_15,home_goals_for_minute_0_15_percentage,home_goals_for_minute_16_30,home_goals_for_minute_16_30_percentage,home_goals_for_minute_31_45,home_goals_for_minute_31_45_percentage,home_goals_for_minute_46_60,home_goals_for_minute_46_60_percentage,home_goals_for_minute_61_75,home_goals_for_minute_61_75_percentage,home_goals_for_minute_76_90,home_goals_for_minute_76_90_percentage,home_goals_for_minute_91_105,home_goals_for_minute_91_105_percentage,home_home_clean_sheets,home_home_draws,home_home_goals_against,home_home_goals_for,home_home_loses,home_home_wins,home_penalties_for,home_total_away_games,home_total_clean_sheets,home_total_draws,home_total_games,home_total_goals_against,home_total_goals_for,home_total_home_games,home_total_loses,home_total_wins
40,1.73,3.60,4.75,1.86,1.92,2.0,554.0,Anderlecht,10/09/2010,144.0,A,0.0,St Truiden,735.0,2010.0,2.0,1.0,0.0,3.0,0.0,1.0,1.0,1.0,2.0,0.4,0.0,0.6,0.2,0.0,0.0,0.2,0.0,0.2,0.4,0.0

## Define indexes and labels

In [29]:
indexes = ['AverageAwayWinOdd', 'AverageDrawWinOdd', 'AverageHomeWinOdd',
               'AvgOver2_5', 'AvgUnder2_5', 'AwayGoals', 'AwayTeamId',
               'AwayTeamId', 'AwayTeamName', 'Date', 'Div',
               'HomeGoals', 'HomeTeamName', 'HomeTeamid', 'Season',
               ]
label = 'FullTimeResult'

In [30]:
print(footballdata_wTeamStats.drop(columns=indexes).dtypes.sort_index().to_string())

FullTimeResult                                  object
away_away_clean_sheets                         float64
away_away_draws                                float64
away_away_goals_against                        float64
away_away_goals_for                            float64
away_away_loses                                float64
away_away_wins                                 float64
away_failed_to_score_away                      float64
away_failed_to_score_home                      float64
away_failed_to_score_total                     float64
away_form_last10games_draws                    float64
away_form_last10games_loses                    float64
away_form_last10games_wins                     float64
away_form_last1games_draws                     float64
away_form_last1games_loses                     float64
away_form_last1games_wins                      float64
away_form_last2games_draws                     float64
away_form_last2games_loses                     float64
away_form_

In [31]:
list(set(footballdata_wTeamStats.columns.to_list())-set(indexes)-set([label]))

['home_goals_against_minute_61_75',
 'home_form_last5games_wins',
 'away_form_last10games_loses',
 'away_home_goals_for',
 'away_goals_against_minute_76_90_percentage',
 'home_goals_for_minute_76_90',
 'away_goals_for_minute_31_45_percentage',
 'away_goals_against_minute_31_45',
 'home_total_clean_sheets',
 'away_failed_to_score_home',
 'home_goals_for_minute_31_45_percentage',
 'away_goals_against_minute_16_30',
 'home_goals_for_minute_16_30',
 'home_goals_for_minute_91_105_percentage',
 'away_failed_to_score_total',
 'home_total_games',
 'away_goals_for_minute_31_45',
 'away_home_draws',
 'home_goals_against_minute_0_15',
 'away_goals_for_minute_76_90',
 'home_away_draws',
 'away_form_last2games_draws',
 'away_goals_for_minute_76_90_percentage',
 'away_away_goals_against',
 'away_goals_for_minute_0_15',
 'home_goals_against_minute_46_60',
 'away_form_last1games_wins',
 'home_form_last1games_loses',
 'home_form_last10games_loses',
 'home_away_goals_against',
 'away_goals_for_minute_16

## Normalize Features (per game)

In [32]:
away_away_games_norm = ['away_away_clean_sheets', 'away_away_draws', 'away_away_goals_against',
                      'away_away_goals_for', 'away_away_loses', 'away_away_wins', 'away_failed_to_score_away']

away_home_games_norm = ['away_failed_to_score_home', 'away_home_clean_sheets',
                       'away_home_draws', 'away_home_goals_against', 'away_home_goals_for',
                       'away_home_loses', 'away_home_wins']

away_total_games_norm = ['away_failed_to_score_total', 'away_goals_against_minute_0_15',
                        'away_goals_against_minute_16_30', 'away_goals_against_minute_31_45',
                        'away_goals_against_minute_46_60', 'away_goals_against_minute_61_75',
                        'away_goals_against_minute_76_90', 'away_goals_against_minute_91_105',
                        'away_goals_for_minute_0_15', 'away_goals_for_minute_16_30',
                        'away_goals_for_minute_31_45', 'away_goals_for_minute_46_60',
                        'away_goals_for_minute_61_75', 'away_goals_for_minute_76_90',
                        'away_goals_for_minute_91_105', 'away_penalties_for', 'away_total_clean_sheets',
                        'away_total_draws', 'away_total_goals_against', 'away_total_goals_for', 'away_total_loses',
                        'away_total_wins']

home_away_games_norm = ['home_away_clean_sheets', 'home_away_draws', 'home_away_goals_against',
                      'home_away_goals_for', 'home_away_loses', 'home_away_wins', 'home_failed_to_score_away']

home_home_games_norm = ['home_failed_to_score_home', 'home_home_clean_sheets',
                       'home_home_draws', 'home_home_goals_against', 'home_home_goals_for',
                       'home_home_loses', 'home_home_wins']

home_total_games_norm = ['home_failed_to_score_total', 'home_goals_against_minute_0_15',
                        'home_goals_against_minute_16_30', 'home_goals_against_minute_31_45',
                        'home_goals_against_minute_46_60', 'home_goals_against_minute_61_75',
                        'home_goals_against_minute_76_90', 'home_goals_against_minute_91_105',
                        'home_goals_for_minute_0_15', 'home_goals_for_minute_16_30',
                        'home_goals_for_minute_31_45', 'home_goals_for_minute_46_60',
                        'home_goals_for_minute_61_75', 'home_goals_for_minute_76_90',
                        'home_goals_for_minute_91_105', 'home_penalties_for', 'home_total_clean_sheets',
                        'home_total_draws', 'home_total_goals_against', 'home_total_goals_for', 'home_total_loses',
                        'home_total_wins']

In [33]:
for feature in away_away_games_norm:
    footballdata_wTeamStats[feature] = footballdata_wTeamStats[feature] / footballdata_wTeamStats['away_total_away_games']
for feature in away_home_games_norm:
    footballdata_wTeamStats[feature] = footballdata_wTeamStats[feature] / footballdata_wTeamStats['away_total_home_games']
for feature in away_total_games_norm:
    footballdata_wTeamStats[feature] = footballdata_wTeamStats[feature] / footballdata_wTeamStats['away_total_games']
    
for feature in home_away_games_norm:
    footballdata_wTeamStats[feature] = footballdata_wTeamStats[feature] / footballdata_wTeamStats['home_total_away_games']
for feature in home_home_games_norm:
    footballdata_wTeamStats[feature] = footballdata_wTeamStats[feature] / footballdata_wTeamStats['home_total_home_games']
for feature in home_total_games_norm:
    footballdata_wTeamStats[feature] = footballdata_wTeamStats[feature] / footballdata_wTeamStats['home_total_games']
    
footballdata_wTeamStats

,home_goals_against_minute_61_75,away_home_goals_for,away_goals_against_minute_76_90_percentage,home_goals_for_minute_76_90,away_goals_for_minute_31_45_percentage,AwayGoals,away_goals_against_minute_31_45,home_total_clean_sheets,away_failed_to_score_home,home_goals_for_minute_31_45_percentage,away_goals_against_minute_16_30,home_goals_for_minute_16_30,home_goals_for_minute_91_105_percentage,away_failed_to_score_total,away_goals_for_minute_31_45,away_home_draws,home_away_draws,home_goals_against_minute_0_15,away_goals_for_minute_76_90,away_goals_for_minute_76_90_percentage,away_away_goals_against,away_goals_for_minute_0_15,home_goals_against_minute_46_60,HomeTeamName,home_away_goals_against,away_goals_for_minute_16_30_percentage,away_away_wins,away_penalties_for,home_goals_for_minute_61_75_percentage,away_home_clean_sheets,home_goals_for_minute_16_30_percentage,away_goals_against_minute_0_15_percentage,home_failed_to_score_home,away_goals_against_minute_31_45_percentage,away_goals_against_minute_16_30_percentage,away_goals_against_minute_76_90,home_failed_to_score_away,away_goals_against_minute_61_75_percentage,away_goals_for_minute_0_15_percentage,away_away_goals_for,away_goals_against_minute_91_105_percentage,home_goals_against_minute_31_45_percentage,away_home_wins,away_goals_for_minute_91_105_percentage,away_goals_against_minute_61_75,Date,away_total_clean_sheets,home_goals_for_minute_0_15,away_goals_for_minute_16_30,home_away_wins,away_total_goals_against,away_away_clean_sheets,home_goals_for_minute_31_45,AvgUnder2_5,home_away_goals_for,home_goals_against_minute_76_90,away_goals_for_minute_46_60_percentage,home_home_loses,home_goals_for_minute_0_15_percentage,Season,home_home_clean_sheets,home_failed_to_score_total,home_penalties_for,AwayTeamId,AverageDrawWinOdd,AwayTeamName,AverageHomeWinOdd,home_away_loses,Div,away_goals_against_minute_91_105,away_goals_against_minute_46_60,home_away_clean_sheets,home_goals_for_minute_76_90_percentage,AverageAwayWinOdd,home_total_goals_for,home_home_goals_for,away_goals_against_minute_46_60_percentage,home_home_wins,away_goals_against_minute_0_15,home_goals_against_minute_16_30,home_goals_against_minute_91_105_percentage,away_total_goals_for,away_home_loses,home_goals_for_minute_61_75,AvgOver2_5,away_goals_for_minute_46_60,away_goals_for_minute_61_75,home_home_draws,home_goals_against_minute_46_60_percentage,FullTimeResult,HomeGoals,home_goals_against_minute_61_75_percentage,home_total_goals_against,away_home_goals_against,home_goals_for_minute_46_60,home_goals_for_minute_46_60_percentage,away_away_loses,home_goals_against_minute_16_30_percentage,away_goals_for_minute_91_105,home_home_goals_against,HomeTeamid,home_goals_for_minute_91_105,home_goals_against_minute_91_105,away_away_draws,away_goals_for_minute_61_75_percentage,home_goals_against_minute_76_90_percentage,away_failed_to_score_away,home_goals_against_minute_31_45,home_goals_against_minute_0_15_percentage,home_total_wins,home_total_draws,home_total_loses,home_total_games,away_total_wins,away_total_draws,away_total_loses,away_total_games,home_total_home_games,home_total_away_games,away_total_home_games,away_total_away_games,away_form_last1games_wins,away_form_last1games_draws,away_form_last1games_loses,away_form_last2games_wins,away_form_last2games_draws,away_form_last2games_loses,away_form_last5games_wins,away_form_last5games_draws,away_form_last5games_loses,away_form_last10games_wins,away_form_last10games_draws,away_form_last10games_loses,home_form_last1games_wins,home_form_last1games_draws,home_form_last1games_loses,home_form_last2games_wins,home_form_last2games_draws,home_form_last2games_loses,home_form_last5games_wins,home_form_last5games_draws,home_form_last5games_loses,home_form_last10games_wins,home_form_last10games_draws,home_form_last10games_loses
40,0.000000,2.666667,0.00,0.000000,9.09,2.0,0.200000,0.600000,0.333333,0.00,0.000000,0.400000,0.00,0.400000,0.200000,0.333333,0.333333,0.400000,0.400000,18.18,0.0000,0.200000,

In [34]:
#TODO: this is based only on home team games (for most cases should be ok)
footballdata_wTeamStats['total_games'] = footballdata_wTeamStats['home_total_games']
footballdata_wTeamStats = \
    footballdata_wTeamStats.drop(columns=['away_total_away_games','away_total_home_games', 'away_total_games',
                                          'home_total_away_games', 'home_total_home_games', 'home_total_games'])
footballdata_wTeamStats

,home_goals_against_minute_61_75,away_home_goals_for,away_goals_against_minute_76_90_percentage,home_goals_for_minute_76_90,away_goals_for_minute_31_45_percentage,AwayGoals,away_goals_against_minute_31_45,home_total_clean_sheets,away_failed_to_score_home,home_goals_for_minute_31_45_percentage,away_goals_against_minute_16_30,home_goals_for_minute_16_30,home_goals_for_minute_91_105_percentage,away_failed_to_score_total,away_goals_for_minute_31_45,away_home_draws,home_away_draws,home_goals_against_minute_0_15,away_goals_for_minute_76_90,away_goals_for_minute_76_90_percentage,away_away_goals_against,away_goals_for_minute_0_15,home_goals_against_minute_46_60,HomeTeamName,home_away_goals_against,away_goals_for_minute_16_30_percentage,away_away_wins,away_penalties_for,home_goals_for_minute_61_75_percentage,away_home_clean_sheets,home_goals_for_minute_16_30_percentage,away_goals_against_minute_0_15_percentage,home_failed_to_score_home,away_goals_against_minute_31_45_percentage,away_goals_against_minute_16_30_percentage,away_goals_against_minute_76_90,home_failed_to_score_away,away_goals_against_minute_61_75_percentage,away_goals_for_minute_0_15_percentage,away_away_goals_for,away_goals_against_minute_91_105_percentage,home_goals_against_minute_31_45_percentage,away_home_wins,away_goals_for_minute_91_105_percentage,away_goals_against_minute_61_75,Date,away_total_clean_sheets,home_goals_for_minute_0_15,away_goals_for_minute_16_30,home_away_wins,away_total_goals_against,away_away_clean_sheets,home_goals_for_minute_31_45,AvgUnder2_5,home_away_goals_for,home_goals_against_minute_76_90,away_goals_for_minute_46_60_percentage,home_home_loses,home_goals_for_minute_0_15_percentage,Season,home_home_clean_sheets,home_failed_to_score_total,home_penalties_for,AwayTeamId,AverageDrawWinOdd,AwayTeamName,AverageHomeWinOdd,home_away_loses,Div,away_goals_against_minute_91_105,away_goals_against_minute_46_60,home_away_clean_sheets,home_goals_for_minute_76_90_percentage,AverageAwayWinOdd,home_total_goals_for,home_home_goals_for,away_goals_against_minute_46_60_percentage,home_home_wins,away_goals_against_minute_0_15,home_goals_against_minute_16_30,home_goals_against_minute_91_105_percentage,away_total_goals_for,away_home_loses,home_goals_for_minute_61_75,AvgOver2_5,away_goals_for_minute_46_60,away_goals_for_minute_61_75,home_home_draws,home_goals_against_minute_46_60_percentage,FullTimeResult,HomeGoals,home_goals_against_minute_61_75_percentage,home_total_goals_against,away_home_goals_against,home_goals_for_minute_46_60,home_goals_for_minute_46_60_percentage,away_away_loses,home_goals_against_minute_16_30_percentage,away_goals_for_minute_91_105,home_home_goals_against,HomeTeamid,home_goals_for_minute_91_105,home_goals_against_minute_91_105,away_away_draws,away_goals_for_minute_61_75_percentage,home_goals_against_minute_76_90_percentage,away_failed_to_score_away,home_goals_against_minute_31_45,home_goals_against_minute_0_15_percentage,home_total_wins,home_total_draws,home_total_loses,away_total_wins,away_total_draws,away_total_loses,away_form_last1games_wins,away_form_last1games_draws,away_form_last1games_loses,away_form_last2games_wins,away_form_last2games_draws,away_form_last2games_loses,away_form_last5games_wins,away_form_last5games_draws,away_form_last5games_loses,away_form_last10games_wins,away_form_last10games_draws,away_form_last10games_loses,home_form_last1games_wins,home_form_last1games_draws,home_form_last1games_loses,home_form_last2games_wins,home_form_last2games_draws,home_form_last2games_loses,home_form_last5games_wins,home_form_last5games_draws,home_form_last5games_loses,home_form_last10games_wins,home_form_last10games_draws,home_form_last10games_loses,total_games
40,0.000000,2.666667,0.00,0.000000,9.09,2.0,0.200000,0.600000,0.333333,0.00,0.000000,0.400000,0.00,0.400000,0.200000,0.333333,0.333333,0.400000,0.400000,18.18,0.0000,0.200000,0.400000,St Truiden,2.000000,0.00,0.5000,0.200000,0.00,0.666667,66.67,0.00,0.0000,100.00,0.00,0.000000,0.66666

## Extract Features

In [35]:
footballdata_wTeamStats.groupby('FullTimeResult').count()

,home_goals_against_minute_61_75,away_home_goals_for,away_goals_against_minute_76_90_percentage,home_goals_for_minute_76_90,away_goals_for_minute_31_45_percentage,AwayGoals,away_goals_against_minute_31_45,home_total_clean_sheets,away_failed_to_score_home,home_goals_for_minute_31_45_percentage,away_goals_against_minute_16_30,home_goals_for_minute_16_30,home_goals_for_minute_91_105_percentage,away_failed_to_score_total,away_goals_for_minute_31_45,away_home_draws,home_away_draws,home_goals_against_minute_0_15,away_goals_for_minute_76_90,away_goals_for_minute_76_90_percentage,away_away_goals_against,away_goals_for_minute_0_15,home_goals_against_minute_46_60,HomeTeamName,home_away_goals_against,away_goals_for_minute_16_30_percentage,away_away_wins,away_penalties_for,home_goals_for_minute_61_75_percentage,away_home_clean_sheets,home_goals_for_minute_16_30_percentage,away_goals_against_minute_0_15_percentage,home_failed_to_score_home,away_goals_against_minute_31_45_percentage,away_goals_against_minute_16_30_percentage,away_goals_against_minute_76_90,home_failed_to_score_away,away_goals_against_minute_61_75_percentage,away_goals_for_minute_0_15_percentage,away_away_goals_for,away_goals_against_minute_91_105_percentage,home_goals_against_minute_31_45_percentage,away_home_wins,away_goals_for_minute_91_105_percentage,away_goals_against_minute_61_75,Date,away_total_clean_sheets,home_goals_for_minute_0_15,away_goals_for_minute_16_30,home_away_wins,away_total_goals_against,away_away_clean_sheets,home_goals_for_minute_31_45,AvgUnder2_5,home_away_goals_for,home_goals_against_minute_76_90,away_goals_for_minute_46_60_percentage,home_home_loses,home_goals_for_minute_0_15_percentage,Season,home_home_clean_sheets,home_failed_to_score_total,home_penalties_for,AwayTeamId,AverageDrawWinOdd,AwayTeamName,AverageHomeWinOdd,home_away_loses,Div,away_goals_against_minute_91_105,away_goals_against_minute_46_60,home_away_clean_sheets,home_goals_for_minute_76_90_percentage,AverageAwayWinOdd,home_total_goals_for,home_home_goals_for,away_goals_against_minute_46_60_percentage,home_home_wins,away_goals_against_minute_0_15,home_goals_against_minute_16_30,home_goals_against_minute_91_105_percentage,away_total_goals_for,away_home_loses,home_goals_for_minute_61_75,AvgOver2_5,away_goals_for_minute_46_60,away_goals_for_minute_61_75,home_home_draws,home_goals_against_minute_46_60_percentage,HomeGoals,home_goals_against_minute_61_75_percentage,home_total_goals_against,away_home_goals_against,home_goals_for_minute_46_60,home_goals_for_minute_46_60_percentage,away_away_loses,home_goals_against_minute_16_30_percentage,away_goals_for_minute_91_105,home_home_goals_against,HomeTeamid,home_goals_for_minute_91_105,home_goals_against_minute_91_105,away_away_draws,away_goals_for_minute_61_75_percentage,home_goals_against_minute_76_90_percentage,away_failed_to_score_away,home_goals_against_minute_31_45,home_goals_against_minute_0_15_percentage,home_total_wins,home_total_draws,home_total_loses,away_total_wins,away_total_draws,away_total_loses,away_form_last1games_wins,away_form_last1games_draws,away_form_last1games_loses,away_form_last2games_wins,away_form_last2games_draws,away_form_last2games_loses,away_form_last5games_wins,away_form_last5games_draws,away_form_last5games_loses,away_form_last10games_wins,away_form_last10games_draws,away_form_last10games_loses,home_form_last1games_wins,home_form_last1games_draws,home_form_last1games_loses,home_form_last2games_wins,home_form_last2games_draws,home_form_last2games_loses,home_form_last5games_wins,home_form_last5games_draws,home_form_last5games_loses,home_form_last10games_wins,home_form_last10games_draws,home_form_last10games_loses,total_games
FullTimeResult,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A,1599,1563,1599,1599,1599,1599,1563,1599,1563,1599,1563,1599,1599,1563,1563,1563,1599,1599,1563,1599,1563,1563,1599,1599,1599,1599,1563,1563,1599

In [36]:
footballdata_wTeamStats = footballdata_wTeamStats.dropna()
footballdata_wTeamStats.groupby('FullTimeResult').count()

,home_goals_against_minute_61_75,away_home_goals_for,away_goals_against_minute_76_90_percentage,home_goals_for_minute_76_90,away_goals_for_minute_31_45_percentage,AwayGoals,away_goals_against_minute_31_45,home_total_clean_sheets,away_failed_to_score_home,home_goals_for_minute_31_45_percentage,away_goals_against_minute_16_30,home_goals_for_minute_16_30,home_goals_for_minute_91_105_percentage,away_failed_to_score_total,away_goals_for_minute_31_45,away_home_draws,home_away_draws,home_goals_against_minute_0_15,away_goals_for_minute_76_90,away_goals_for_minute_76_90_percentage,away_away_goals_against,away_goals_for_minute_0_15,home_goals_against_minute_46_60,HomeTeamName,home_away_goals_against,away_goals_for_minute_16_30_percentage,away_away_wins,away_penalties_for,home_goals_for_minute_61_75_percentage,away_home_clean_sheets,home_goals_for_minute_16_30_percentage,away_goals_against_minute_0_15_percentage,home_failed_to_score_home,away_goals_against_minute_31_45_percentage,away_goals_against_minute_16_30_percentage,away_goals_against_minute_76_90,home_failed_to_score_away,away_goals_against_minute_61_75_percentage,away_goals_for_minute_0_15_percentage,away_away_goals_for,away_goals_against_minute_91_105_percentage,home_goals_against_minute_31_45_percentage,away_home_wins,away_goals_for_minute_91_105_percentage,away_goals_against_minute_61_75,Date,away_total_clean_sheets,home_goals_for_minute_0_15,away_goals_for_minute_16_30,home_away_wins,away_total_goals_against,away_away_clean_sheets,home_goals_for_minute_31_45,AvgUnder2_5,home_away_goals_for,home_goals_against_minute_76_90,away_goals_for_minute_46_60_percentage,home_home_loses,home_goals_for_minute_0_15_percentage,Season,home_home_clean_sheets,home_failed_to_score_total,home_penalties_for,AwayTeamId,AverageDrawWinOdd,AwayTeamName,AverageHomeWinOdd,home_away_loses,Div,away_goals_against_minute_91_105,away_goals_against_minute_46_60,home_away_clean_sheets,home_goals_for_minute_76_90_percentage,AverageAwayWinOdd,home_total_goals_for,home_home_goals_for,away_goals_against_minute_46_60_percentage,home_home_wins,away_goals_against_minute_0_15,home_goals_against_minute_16_30,home_goals_against_minute_91_105_percentage,away_total_goals_for,away_home_loses,home_goals_for_minute_61_75,AvgOver2_5,away_goals_for_minute_46_60,away_goals_for_minute_61_75,home_home_draws,home_goals_against_minute_46_60_percentage,HomeGoals,home_goals_against_minute_61_75_percentage,home_total_goals_against,away_home_goals_against,home_goals_for_minute_46_60,home_goals_for_minute_46_60_percentage,away_away_loses,home_goals_against_minute_16_30_percentage,away_goals_for_minute_91_105,home_home_goals_against,HomeTeamid,home_goals_for_minute_91_105,home_goals_against_minute_91_105,away_away_draws,away_goals_for_minute_61_75_percentage,home_goals_against_minute_76_90_percentage,away_failed_to_score_away,home_goals_against_minute_31_45,home_goals_against_minute_0_15_percentage,home_total_wins,home_total_draws,home_total_loses,away_total_wins,away_total_draws,away_total_loses,away_form_last1games_wins,away_form_last1games_draws,away_form_last1games_loses,away_form_last2games_wins,away_form_last2games_draws,away_form_last2games_loses,away_form_last5games_wins,away_form_last5games_draws,away_form_last5games_loses,away_form_last10games_wins,away_form_last10games_draws,away_form_last10games_loses,home_form_last1games_wins,home_form_last1games_draws,home_form_last1games_loses,home_form_last2games_wins,home_form_last2games_draws,home_form_last2games_loses,home_form_last5games_wins,home_form_last5games_draws,home_form_last5games_loses,home_form_last10games_wins,home_form_last10games_draws,home_form_last10games_loses,total_games
FullTimeResult,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A,1563,1563,1563,1563,1563,1563,1563,1563,1563,1563,1563,1563,1563,1563,1563,1563,1563,1563,1563,1563,1563,1563,1563,1563,1563,1563,1563,1563,1563

In [37]:
#y = np.array([0 if label=='H' else 1 if label=='D' else 2 for label in y])
#y = np.array([0 if label=='H' else 1 for label in y])
footballdata_wTeamStats['target'] = footballdata_wTeamStats[label].apply(lambda x: 0 if x=='H' else 1)

df_train, df_test \
    = train_test_split(footballdata_wTeamStats, test_size=0.2, random_state=1)

df_train, df_val \
    = train_test_split(df_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

df_train.head(5)

<ipython-input-37-2f3e3fdb911e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  footballdata_wTeamStats['target'] = footballdata_wTeamStats[label].apply(lambda x: 0 if x=='H' else 1)


,home_goals_against_minute_61_75,away_home_goals_for,away_goals_against_minute_76_90_percentage,home_goals_for_minute_76_90,away_goals_for_minute_31_45_percentage,AwayGoals,away_goals_against_minute_31_45,home_total_clean_sheets,away_failed_to_score_home,home_goals_for_minute_31_45_percentage,away_goals_against_minute_16_30,home_goals_for_minute_16_30,home_goals_for_minute_91_105_percentage,away_failed_to_score_total,away_goals_for_minute_31_45,away_home_draws,home_away_draws,home_goals_against_minute_0_15,away_goals_for_minute_76_90,away_goals_for_minute_76_90_percentage,away_away_goals_against,away_goals_for_minute_0_15,home_goals_against_minute_46_60,HomeTeamName,home_away_goals_against,away_goals_for_minute_16_30_percentage,away_away_wins,away_penalties_for,home_goals_for_minute_61_75_percentage,away_home_clean_sheets,home_goals_for_minute_16_30_percentage,away_goals_against_minute_0_15_percentage,home_failed_to_score_home,away_goals_against_minute_31_45_percentage,away_goals_against_minute_16_30_percentage,away_goals_against_minute_76_90,home_failed_to_score_away,away_goals_against_minute_61_75_percentage,away_goals_for_minute_0_15_percentage,away_away_goals_for,away_goals_against_minute_91_105_percentage,home_goals_against_minute_31_45_percentage,away_home_wins,away_goals_for_minute_91_105_percentage,away_goals_against_minute_61_75,Date,away_total_clean_sheets,home_goals_for_minute_0_15,away_goals_for_minute_16_30,home_away_wins,away_total_goals_against,away_away_clean_sheets,home_goals_for_minute_31_45,AvgUnder2_5,home_away_goals_for,home_goals_against_minute_76_90,away_goals_for_minute_46_60_percentage,home_home_loses,home_goals_for_minute_0_15_percentage,Season,home_home_clean_sheets,home_failed_to_score_total,home_penalties_for,AwayTeamId,AverageDrawWinOdd,AwayTeamName,AverageHomeWinOdd,home_away_loses,Div,away_goals_against_minute_91_105,away_goals_against_minute_46_60,home_away_clean_sheets,home_goals_for_minute_76_90_percentage,AverageAwayWinOdd,home_total_goals_for,home_home_goals_for,away_goals_against_minute_46_60_percentage,home_home_wins,away_goals_against_minute_0_15,home_goals_against_minute_16_30,home_goals_against_minute_91_105_percentage,away_total_goals_for,away_home_loses,home_goals_for_minute_61_75,AvgOver2_5,away_goals_for_minute_46_60,away_goals_for_minute_61_75,home_home_draws,home_goals_against_minute_46_60_percentage,FullTimeResult,HomeGoals,home_goals_against_minute_61_75_percentage,home_total_goals_against,away_home_goals_against,home_goals_for_minute_46_60,home_goals_for_minute_46_60_percentage,away_away_loses,home_goals_against_minute_16_30_percentage,away_goals_for_minute_91_105,home_home_goals_against,HomeTeamid,home_goals_for_minute_91_105,home_goals_against_minute_91_105,away_away_draws,away_goals_for_minute_61_75_percentage,home_goals_against_minute_76_90_percentage,away_failed_to_score_away,home_goals_against_minute_31_45,home_goals_against_minute_0_15_percentage,home_total_wins,home_total_draws,home_total_loses,away_total_wins,away_total_draws,away_total_loses,away_form_last1games_wins,away_form_last1games_draws,away_form_last1games_loses,away_form_last2games_wins,away_form_last2games_draws,away_form_last2games_loses,away_form_last5games_wins,away_form_last5games_draws,away_form_last5games_loses,away_form_last10games_wins,away_form_last10games_draws,away_form_last10games_loses,home_form_last1games_wins,home_form_last1games_draws,home_form_last1games_loses,home_form_last2games_wins,home_form_last2games_draws,home_form_last2games_loses,home_form_last5games_wins,home_form_last5games_draws,home_form_last5games_loses,home_form_last10games_wins,home_form_last10games_draws,home_form_last10games_loses,total_games,target
1729,0.217391,1.583333,16.67,0.173913,13.79,1.0,0.130435,0.217391,0.083333,9.76,0.217391,0.304348,4.88,0.260870,0.173913,0.250000,0.181818,0.304348,0.260870,20.69,1.727273,0.173913,0.130435,AZ Alkmaar,1.545455,17.24,0.181818,0.000000,26.83,0.416667,17.07,10.00,0.333333,10.00,16

In [38]:
print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(3120, 141)
(1041, 141)
(1041, 141)


In [39]:
scaler = StandardScaler()
# scale based only on the training set
scaler.fit(df_train[list(set(df_train.columns.to_list())-set(indexes)-set([label])-set(['target']))])

X_train = scaler.transform(df_train[list(set(df_train.columns.to_list())-set(indexes)-set([label])-set(['target']))])
X_val = scaler.transform(df_val[list(set(df_val.columns.to_list())-set(indexes)-set([label])-set(['target']))])
X_test = scaler.transform(df_test[list(set(df_test.columns.to_list())-set(indexes)-set([label])-set(['target']))])

y_train = df_train['target'].values
y_train = y_train.reshape((y_train.shape[0],))

y_val = df_val['target'].values
y_val = y_val.reshape((y_val.shape[0],))

y_test = df_test['target'].values
y_test = y_test.reshape((y_test.shape[0],))

In [40]:
X_train

array([[-0.29092013,  2.06768304, -1.24210314, ..., -0.16322959,
         1.0838147 ,  1.1457737 ],
       [-0.05792076,  0.32512576, -0.02954252, ..., -1.02598032,
         1.30162627, -0.80890034],
       [-0.52104295, -0.7204086 ,  0.077614  , ...,  0.29690413,
        -0.61324475, -0.27580742],
       ...,
       [-0.41518645, -1.41743152, -0.14008293, ...,  0.72022715,
        -0.18296669, -0.27580742],
       [ 0.78823482, -0.02338569,  0.31335835, ..., -0.81710383,
        -0.33129235,  0.43498314],
       [-0.31311054, -0.02338569,  1.1277479 , ..., -0.45902984,
         0.83794152, -0.27580742]])

In [41]:
y_train

array([0, 1, 1, ..., 0, 0, 1], dtype=int64)

In [42]:
print ('The first element of X is: ', X_train[0])

The first element of X is:  [-0.29092013  2.06768304 -1.24210314 -0.36397332  0.18513043 -0.17773417
 -0.23326043  1.78945711 -0.60038896 -0.69778363 -0.13108483 -0.68204727
 -0.15172414  0.15826155 -0.37049642 -0.70088041 -0.66016594  0.04821596
  0.12313507  0.75566204  0.05516804 -0.39753601  0.1669363   0.2178859
  0.44016697 -0.37301523  0.2376032   0.02170702  0.9792001  -0.17584473
 -0.43524374 -0.76418862 -1.16054471 -0.36691237 -0.98098297  0.1486503
  0.08843379 -0.15207034  0.49237758 -0.09704313  0.05110974  0.58048972
 -0.98252774 -0.85568263 -0.85085736  0.1612255  -0.39748897 -0.22887745
 -0.46633304  0.42149256 -0.53861034 -1.18594365 -1.25914928  1.15455106
 -0.37195321  0.71139522  0.416913   -0.47657312 -0.11187617  0.54763663
 -0.61284496  1.401635   -0.44200072 -1.16768239 -0.56549161 -0.47340994
 -1.0852145  -0.33367843  1.26180276 -0.72664857  0.80936005 -0.28866352
 -0.05986574 -0.70798257 -1.16099624 -0.57788605 -1.46334502  1.52307917
 -0.75538321  0.38276268 

In [43]:
print ('The first element of y is: ', y_train[0])
print ('The last element of y is: ', y_train[-1])

The first element of y is:  0
The last element of y is:  1


In [44]:
print ('The shape of X is: ' + str(X_train.shape))
print ('The shape of y is: ' + str(y_train.shape))

The shape of X is: (3120, 125)
The shape of y is: (3120,)


# ML Algorithms

## MLP

In [79]:
model, y_train_pred_probs, y_test_pred_probs, y_train_pred, y_test_pred = \
    MLP(np.concatenate([X_train,X_val]), X_test, np.concatenate([y_train,y_val]), y_test,
        layers_config=[(10,'relu'),(10,'relu'),(2,'linear')], task='classification', epochs=500, batch_size=-1)

Epoch 1/500
1/1 [==============================] - 0s 235ms/step - loss: 3.9264
Epoch 2/500
1/1 [==============================] - 0s 6ms/step - loss: 3.8852
Epoch 3/500
1/1 [==============================] - 0s 7ms/step - loss: 3.8449
Epoch 4/500
1/1 [==============================] - 0s 5ms/step - loss: 3.8053
Epoch 5/500
1/1 [==============================] - 0s 5ms/step - loss: 3.7665
Epoch 6/500
1/1 [==============================] - 0s 7ms/step - loss: 3.7284
Epoch 7/500
1/1 [==============================] - 0s 5ms/step - loss: 3.6909
Epoch 8/500
1/1 [==============================] - 0s 7ms/step - loss: 3.6542
Epoch 9/500
1/1 [==============================] - 0s 5ms/step - loss: 3.6181
Epoch 10/500
1/1 [==============================] - 0s 7ms/step - loss: 3.5828
Epoch 11/500
1/1 [==============================] - 0s 6ms/step - loss: 3.5480
Epoch 12/500
1/1 [==============================] - 0s 7ms/step - loss: 3.5138
Epoch 13/500
1/1 [==============================] - 0s 5ms/

1/1 [==============================] - 0s 5ms/step - loss: 1.7202
Epoch 105/500
1/1 [==============================] - 0s 5ms/step - loss: 1.7098
Epoch 106/500
1/1 [==============================] - 0s 5ms/step - loss: 1.6994
Epoch 107/500
1/1 [==============================] - 0s 6ms/step - loss: 1.6892
Epoch 108/500
1/1 [==============================] - 0s 5ms/step - loss: 1.6792
Epoch 109/500
1/1 [==============================] - 0s 5ms/step - loss: 1.6692
Epoch 110/500
1/1 [==============================] - 0s 5ms/step - loss: 1.6594
Epoch 111/500
1/1 [==============================] - 0s 5ms/step - loss: 1.6496
Epoch 112/500
1/1 [==============================] - 0s 5ms/step - loss: 1.6400
Epoch 113/500
1/1 [==============================] - 0s 4ms/step - loss: 1.6305
Epoch 114/500
1/1 [==============================] - 0s 5ms/step - loss: 1.6212
Epoch 115/500
1/1 [==============================] - 0s 5ms/step - loss: 1.6119
Epoch 116/500
1/1 [==============================] - 0

1/1 [==============================] - 0s 5ms/step - loss: 1.0850
Epoch 207/500
1/1 [==============================] - 0s 6ms/step - loss: 1.0816
Epoch 208/500
1/1 [==============================] - 0s 5ms/step - loss: 1.0782
Epoch 209/500
1/1 [==============================] - 0s 5ms/step - loss: 1.0748
Epoch 210/500
1/1 [==============================] - 0s 6ms/step - loss: 1.0715
Epoch 211/500
1/1 [==============================] - 0s 4ms/step - loss: 1.0682
Epoch 212/500
1/1 [==============================] - 0s 6ms/step - loss: 1.0649
Epoch 213/500
1/1 [==============================] - 0s 6ms/step - loss: 1.0617
Epoch 214/500
1/1 [==============================] - 0s 5ms/step - loss: 1.0585
Epoch 215/500
1/1 [==============================] - 0s 6ms/step - loss: 1.0553
Epoch 216/500
1/1 [==============================] - 0s 5ms/step - loss: 1.0522
Epoch 217/500
1/1 [==============================] - 0s 7ms/step - loss: 1.0490
Epoch 218/500
1/1 [==============================] - 0

1/1 [==============================] - 0s 5ms/step - loss: 0.8552
Epoch 309/500
1/1 [==============================] - 0s 6ms/step - loss: 0.8538
Epoch 310/500
1/1 [==============================] - 0s 4ms/step - loss: 0.8524
Epoch 311/500
1/1 [==============================] - 0s 6ms/step - loss: 0.8509
Epoch 312/500
1/1 [==============================] - 0s 6ms/step - loss: 0.8495
Epoch 313/500
1/1 [==============================] - 0s 5ms/step - loss: 0.8481
Epoch 314/500
1/1 [==============================] - 0s 5ms/step - loss: 0.8467
Epoch 315/500
1/1 [==============================] - 0s 6ms/step - loss: 0.8453
Epoch 316/500
1/1 [==============================] - 0s 5ms/step - loss: 0.8439
Epoch 317/500
1/1 [==============================] - 0s 4ms/step - loss: 0.8426
Epoch 318/500
1/1 [==============================] - 0s 6ms/step - loss: 0.8412
Epoch 319/500
1/1 [==============================] - 0s 5ms/step - loss: 0.8399
Epoch 320/500
1/1 [==============================] - 0

1/1 [==============================] - 0s 6ms/step - loss: 0.7497
Epoch 411/500
1/1 [==============================] - 0s 4ms/step - loss: 0.7490
Epoch 412/500
1/1 [==============================] - 0s 5ms/step - loss: 0.7482
Epoch 413/500
1/1 [==============================] - 0s 4ms/step - loss: 0.7475
Epoch 414/500
1/1 [==============================] - 0s 5ms/step - loss: 0.7468
Epoch 415/500
1/1 [==============================] - 0s 6ms/step - loss: 0.7461
Epoch 416/500
1/1 [==============================] - 0s 7ms/step - loss: 0.7454
Epoch 417/500
1/1 [==============================] - 0s 5ms/step - loss: 0.7447
Epoch 418/500
1/1 [==============================] - 0s 5ms/step - loss: 0.7441
Epoch 419/500
1/1 [==============================] - 0s 5ms/step - loss: 0.7434
Epoch 420/500
1/1 [==============================] - 0s 5ms/step - loss: 0.7427
Epoch 421/500
1/1 [==============================] - 0s 5ms/step - loss: 0.7420
Epoch 422/500
1/1 [==============================] - 0

In [80]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 L1 (Dense)                  (None, 10)                1260      
                                                                 
 L2 (Dense)                  (None, 10)                110       
                                                                 
 L3 (Dense)                  (None, 2)                 22        
                                                                 
Total params: 1,392
Trainable params: 1,392
Non-trainable params: 0
_________________________________________________________________


In [81]:
[layer1, layer2, layer3] = model.layers
W1,b1 = layer1.get_weights()
W2,b2 = layer2.get_weights()
W3,b3 = layer3.get_weights()

In [82]:
cm_train = confusion_matrix(y_train_pred, np.concatenate([y_train,y_val]))
cm_train

array([[1073,  599],
       [ 855, 1634]], dtype=int64)

In [83]:
cm_test = confusion_matrix(y_test_pred, y_test)
cm_test

array([[259, 146],
       [232, 404]], dtype=int64)

In [84]:
for i in range(10):
    print( f"{y_train_pred_probs[-i]}, predicted_category: {np.argmax(y_train_pred_probs[-i])}, category: {y_train[-i]}")

[0.5742105 0.4257895], predicted_category: 0, category: 0
[0.31961894 0.680381  ], predicted_category: 1, category: 1
[0.7426247  0.25737527], predicted_category: 0, category: 0
[0.47576195 0.52423805], predicted_category: 1, category: 0
[0.48025385 0.5197462 ], predicted_category: 1, category: 1
[0.4441932  0.55580676], predicted_category: 1, category: 1
[0.30773166 0.6922683 ], predicted_category: 1, category: 1
[0.6229818  0.37701824], predicted_category: 0, category: 1
[0.46533182 0.5346681 ], predicted_category: 1, category: 0
[0.17236869 0.82763135], predicted_category: 1, category: 1


In [85]:
for i in range(10):
    print( f"{y_test_pred_probs[-i]}, predicted_category: {np.argmax(y_test_pred_probs[-i])}, category: {y_test[-i]}")

[0.505638 0.494362], predicted_category: 0, category: 1
[0.33388305 0.66611695], predicted_category: 1, category: 0
[0.43162224 0.5683777 ], predicted_category: 1, category: 1
[0.6233574 0.3766426], predicted_category: 0, category: 0
[0.53845185 0.4615482 ], predicted_category: 0, category: 0
[0.22492851 0.7750715 ], predicted_category: 1, category: 0
[0.25005725 0.7499427 ], predicted_category: 1, category: 0
[0.3752126 0.6247874], predicted_category: 1, category: 1
[0.33740136 0.66259855], predicted_category: 1, category: 0
[0.72240967 0.2775904 ], predicted_category: 0, category: 0


In [86]:
print('Train accuracy: '+ str(accuracy(cm_train)))
print('Test accuracy: '+ str(accuracy(cm_test)))

Train accuracy: 0.6505647680845951
Test accuracy: 0.6368876080691642


In [87]:
df_test['predictions'] = y_test_pred
df_test

<ipython-input-87-4a0c088fe1b7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['predictions'] = y_test_pred


,home_goals_against_minute_61_75,away_home_goals_for,away_goals_against_minute_76_90_percentage,home_goals_for_minute_76_90,away_goals_for_minute_31_45_percentage,AwayGoals,away_goals_against_minute_31_45,home_total_clean_sheets,away_failed_to_score_home,home_goals_for_minute_31_45_percentage,away_goals_against_minute_16_30,home_goals_for_minute_16_30,home_goals_for_minute_91_105_percentage,away_failed_to_score_total,away_goals_for_minute_31_45,away_home_draws,home_away_draws,home_goals_against_minute_0_15,away_goals_for_minute_76_90,away_goals_for_minute_76_90_percentage,away_away_goals_against,away_goals_for_minute_0_15,home_goals_against_minute_46_60,HomeTeamName,home_away_goals_against,away_goals_for_minute_16_30_percentage,away_away_wins,away_penalties_for,home_goals_for_minute_61_75_percentage,away_home_clean_sheets,home_goals_for_minute_16_30_percentage,away_goals_against_minute_0_15_percentage,home_failed_to_score_home,away_goals_against_minute_31_45_percentage,away_goals_against_minute_16_30_percentage,away_goals_against_minute_76_90,home_failed_to_score_away,away_goals_against_minute_61_75_percentage,away_goals_for_minute_0_15_percentage,away_away_goals_for,away_goals_against_minute_91_105_percentage,home_goals_against_minute_31_45_percentage,away_home_wins,away_goals_for_minute_91_105_percentage,away_goals_against_minute_61_75,Date,away_total_clean_sheets,home_goals_for_minute_0_15,away_goals_for_minute_16_30,home_away_wins,away_total_goals_against,away_away_clean_sheets,home_goals_for_minute_31_45,AvgUnder2_5,home_away_goals_for,home_goals_against_minute_76_90,away_goals_for_minute_46_60_percentage,home_home_loses,home_goals_for_minute_0_15_percentage,Season,home_home_clean_sheets,home_failed_to_score_total,home_penalties_for,AwayTeamId,AverageDrawWinOdd,AwayTeamName,AverageHomeWinOdd,home_away_loses,Div,away_goals_against_minute_91_105,away_goals_against_minute_46_60,home_away_clean_sheets,home_goals_for_minute_76_90_percentage,AverageAwayWinOdd,home_total_goals_for,home_home_goals_for,away_goals_against_minute_46_60_percentage,home_home_wins,away_goals_against_minute_0_15,home_goals_against_minute_16_30,home_goals_against_minute_91_105_percentage,away_total_goals_for,away_home_loses,home_goals_for_minute_61_75,AvgOver2_5,away_goals_for_minute_46_60,away_goals_for_minute_61_75,home_home_draws,home_goals_against_minute_46_60_percentage,FullTimeResult,HomeGoals,home_goals_against_minute_61_75_percentage,home_total_goals_against,away_home_goals_against,home_goals_for_minute_46_60,home_goals_for_minute_46_60_percentage,away_away_loses,home_goals_against_minute_16_30_percentage,away_goals_for_minute_91_105,home_home_goals_against,HomeTeamid,home_goals_for_minute_91_105,home_goals_against_minute_91_105,away_away_draws,away_goals_for_minute_61_75_percentage,home_goals_against_minute_76_90_percentage,away_failed_to_score_away,home_goals_against_minute_31_45,home_goals_against_minute_0_15_percentage,home_total_wins,home_total_draws,home_total_loses,away_total_wins,away_total_draws,away_total_loses,away_form_last1games_wins,away_form_last1games_draws,away_form_last1games_loses,away_form_last2games_wins,away_form_last2games_draws,away_form_last2games_loses,away_form_last5games_wins,away_form_last5games_draws,away_form_last5games_loses,away_form_last10games_wins,away_form_last10games_draws,away_form_last10games_loses,home_form_last1games_wins,home_form_last1games_draws,home_form_last1games_loses,home_form_last2games_wins,home_form_last2games_draws,home_form_last2games_loses,home_form_last5games_wins,home_form_last5games_draws,home_form_last5games_loses,home_form_last10games_wins,home_form_last10games_draws,home_form_last10games_loses,total_games,target,predictions
1601,0.200000,1.400000,17.24,0.200000,19.05,0.0,0.157895,0.150000,0.300000,10.00,0.315789,0.150000,0.00,0.421053,0.210526,0.400000,0.272727,0.050000,0.157895,14.29,2.222222,0.315789,0.150000,Lokeren,1.454545,9.52,0.000000,0.105263,30.00,0.200000,15.00,10.34,0.333333

In [70]:
(df_test['target'] == df_test['predictions']).mean()

0.6311239193083573

## GBT

In [141]:
model, y_train_pred_probs, y_test_pred_probs, y_train_pred, y_test_pred = \
    GBT(np.concatenate([X_train,X_val]), X_test, np.concatenate([y_train,y_val]), y_test,
        max_depth=3, n_iter_no_change=None, verbose=1)

      Iter       Train Loss   Remaining Time 
         1           1.3619            6.50s
         2           1.3459            6.39s
         3           1.3314            6.15s
         4           1.3195            6.36s
         5           1.3085            6.27s
         6           1.2988            6.11s
         7           1.2896            6.24s
         8           1.2815            6.21s
         9           1.2740            6.27s
        10           1.2674            6.21s
        20           1.2186            5.24s
        30           1.1854            4.37s
        40           1.1616            3.96s
        50           1.1401            3.40s
        60           1.1203            2.74s
        70           1.1024            2.07s
        80           1.0847            1.42s
        90           1.0721            0.71s
       100           1.0557            0.00s


In [142]:
cm_train = confusion_matrix(y_train_pred, np.concatenate([y_train,y_val]))
cm_train

array([[1324,  380],
       [ 604, 1853]], dtype=int64)

In [143]:
cm_test = confusion_matrix(y_test_pred, y_test)
cm_test

array([[253, 159],
       [238, 391]], dtype=int64)

In [144]:
print('Train accuracy: '+ str(accuracy(cm_train)))
print('Test accuracy: '+ str(accuracy(cm_test)))

Train accuracy: 0.7635183850036049
Test accuracy: 0.6186359269932757


In [ ]:
df_test['predictions'] = y_test_pred
df_test

### Test changing only max_depth

In [56]:
max_val_score = 0
iterations_wo_update = 0
for max_depth in range(125):
    max_depth += 1 # start on 1
    print(f"Testing Max Depth: {max_depth}")
    gbtClassifier = GradientBoostingClassifier(max_depth = max_depth, n_iter_no_change=7, verbose=1).fit(X_train, y_train)
    val_score = gbtClassifier.score(X_val, y_val)
    if val_score > max_val_score:
        iterations_wo_update = 0
        best_max_depth = max_depth
        max_val_score = val_score
        best_model = gbtClassifier
        print(f"New best max depth: {best_max_depth}, validation score: {max_val_score}")
    else:
        iterations_wo_update +=1
        if iterations_wo_update >= 10:
            print("No improvements on the last 10 iterations, stopping here...")
            break
            
gbtClassifier = best_model

Testing Max Depth: 1
      Iter       Train Loss   Remaining Time 
         1           1.0487            5.04s
         2           1.0434            4.24s
         3           1.0387            3.94s
         4           1.0342            3.77s
         5           1.0302            3.72s
         6           1.0265            3.62s
         7           1.0229            3.54s
         8           1.0196            3.49s
         9           1.0164            3.41s
        10           1.0135            3.35s
        20           0.9918            2.86s
        30           0.9782            2.48s
        40           0.9689            2.15s
        50           0.9617            1.79s
        60           0.9560            1.44s
        70           0.9509            1.08s
New best max depth: 1, validation score: 0.5024015369836695
Testing Max Depth: 2
      Iter       Train Loss   Remaining Time 
         1           1.0431            6.59s
         2           1.0335            6.

In [57]:
best_model.score(X_train, y_train)

0.5634615384615385

In [58]:
best_model.score(X_test, y_test)

0.5091258405379443

In [59]:
y_train_pred = gbtClassifier.predict(X_train)
y_test_pred = gbtClassifier.predict(X_test)

In [60]:
cm_train = confusion_matrix(y_train_pred, y_train)
cm_train

array([[1252,  493,  484],
       [   4,   30,    3],
       [ 197,  181,  476]], dtype=int64)

In [61]:
cm_test = confusion_matrix(y_test_pred, y_test)
cm_test

array([[397, 190, 160],
       [  5,   3,   5],
       [ 89,  62, 130]], dtype=int64)

### Test changing also n_estimators

In [81]:
gbtClassifier = GradientBoostingClassifier(n_estimators=20, verbose=1).fit(X_train, y_train)
print(gbtClassifier.score(X_train, y_train))
gbtClassifier.score(X_test, y_test)

      Iter       Train Loss   Remaining Time 
         1           1.0605           19.28s
         2           1.0529           18.00s
         3           1.0463           17.03s
         4           1.0409           16.72s
         5           1.0361           15.48s
         6           1.0319           14.35s
         7           1.0284           13.24s
         8           1.0252           12.17s
         9           1.0225           11.11s
        10           1.0201           10.07s
        20           1.0053            0.00s
0.5003111145679396


0.49848343443770415

In [82]:
max_val_score = 0
for n_estimators in range(5,100,5):
    print(f"Testing N Estimators: {n_estimators}")
    iterations_wo_update = 0
    for max_depth in range(125):
        max_depth += 1 # start on 1
        gbtClassifier = GradientBoostingClassifier(max_depth = max_depth, n_estimators=n_estimators).fit(X_train, y_train)
        val_score = gbtClassifier.score(X_val, y_val)
        if val_score > max_val_score:
            iterations_wo_update = 0
            best_max_depth = max_depth
            max_val_score = val_score
            best_model = gbtClassifier
            print(f"New best max depth and N estimators: ({best_max_depth}, {n_estimators}), validation score: {max_val_score}")
        else:
            iterations_wo_update +=1
            if iterations_wo_update >= 10:
                print("No improvements on the last 10 iterations, moving on...")
                break
            
gbtClassifier = best_model

Testing N Estimators: 5
New best max depth and N estimators: (1, 5), validation score: 0.4498366775548297
New best max depth and N estimators: (2, 5), validation score: 0.46978534764349045
New best max depth and N estimators: (3, 5), validation score: 0.47316845543630426
New best max depth and N estimators: (4, 5), validation score: 0.4788847410172655
New best max depth and N estimators: (5, 5), validation score: 0.4794680354643024
No improvements on the last 10 iterations, moving on...
Testing N Estimators: 10
New best max depth and N estimators: (3, 10), validation score: 0.4836677554829678
New best max depth and N estimators: (4, 10), validation score: 0.4841343910405973
New best max depth and N estimators: (5, 10), validation score: 0.4849510032664489
No improvements on the last 10 iterations, moving on...
Testing N Estimators: 15
New best max depth and N estimators: (3, 15), validation score: 0.48611759216052264
New best max depth and N estimators: (4, 15), validation score: 0.486

In [83]:
gbtClassifier.score(X_train, y_train)

0.5594617717974644

In [84]:
gbtClassifier.score(X_test, y_test)

0.4996500233317779

In [85]:
y_train_pred = gbtClassifier.predict(X_train)
y_test_pred = gbtClassifier.predict(X_test)

In [86]:
cm_train = confusion_matrix(y_train_pred, y_train)
cm_train

array([[10198,  4701,  3894],
       [   45,   698,    51],
       [ 1272,  1365,  3490]], dtype=int64)

In [87]:
cm_test = confusion_matrix(y_test_pred, y_test)
cm_test

array([[3233, 1595, 1495],
       [  61,   50,   37],
       [ 526,  575, 1000]], dtype=int64)

## Random Forest

In [147]:
model, y_train_pred_probs, y_test_pred_probs, y_train_pred, y_test_pred = \
    RandomForest(np.concatenate([X_train,X_val]), X_test, np.concatenate([y_train,y_val]), y_test, verbose=1)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


In [148]:
cm_train = confusion_matrix(y_train_pred, np.concatenate([y_train,y_val]))
cm_train

array([[1928,    0],
       [   0, 2233]], dtype=int64)

In [149]:
cm_test = confusion_matrix(y_test_pred, y_test)
cm_test

array([[262, 153],
       [229, 397]], dtype=int64)

In [150]:
print('Train accuracy: '+ str(accuracy(cm_train)))
print('Test accuracy: '+ str(accuracy(cm_test)))

Train accuracy: 1.0
Test accuracy: 0.633045148895293


In [151]:
df_test['predictions'] = y_test_pred
df_test

<ipython-input-151-4a0c088fe1b7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['predictions'] = y_test_pred


,home_goals_for_minute_31_45_percentage,away_total_goals_for,away_total_goals_against,away_goals_against_minute_31_45,Date,FullTimeResult,away_failed_to_score_total,away_away_loses,home_goals_against_minute_61_75_percentage,Div,AverageAwayWinOdd,home_goals_against_minute_46_60_percentage,AvgOver2_5,away_goals_against_minute_91_105,home_failed_to_score_total,home_goals_against_minute_76_90_percentage,home_goals_against_minute_31_45_percentage,away_goals_against_minute_0_15_percentage,away_away_goals_for,away_away_goals_against,AvgUnder2_5,away_goals_for_minute_91_105_percentage,home_goals_against_minute_61_75,away_goals_for_minute_46_60,away_goals_for_minute_31_45,home_goals_against_minute_16_30_percentage,home_goals_against_minute_31_45,home_away_goals_against,away_goals_for_minute_31_45_percentage,away_goals_against_minute_46_60_percentage,home_goals_for_minute_46_60_percentage,away_penalties_for,home_failed_to_score_away,away_home_draws,home_penalties_for,home_away_draws,away_goals_for_minute_76_90,away_away_draws,home_goals_for_minute_0_15_percentage,home_goals_for_minute_76_90_percentage,home_goals_for_minute_46_60,away_goals_against_minute_61_75_percentage,home_total_goals_against,away_goals_against_minute_61_75,home_goals_against_minute_16_30,away_goals_for_minute_16_30_percentage,home_failed_to_score_home,home_away_loses,away_goals_against_minute_16_30_percentage,away_failed_to_score_away,AverageDrawWinOdd,AwayTeamId,away_goals_against_minute_76_90_percentage,away_home_loses,home_goals_for_minute_31_45,home_goals_for_minute_91_105,HomeGoals,away_total_clean_sheets,home_home_loses,away_goals_against_minute_16_30,away_home_clean_sheets,home_goals_for_minute_16_30_percentage,away_goals_for_minute_0_15_percentage,away_goals_for_minute_91_105,home_away_goals_for,away_goals_against_minute_91_105_percentage,home_home_clean_sheets,home_goals_against_minute_0_15_percentage,Season,home_goals_for_minute_76_90,away_goals_against_minute_76_90,away_goals_against_minute_31_45_percentage,home_goals_for_minute_0_15,home_home_goals_against,AwayGoals,away_goals_for_minute_61_75,home_goals_against_minute_46_60,home_goals_for_minute_61_75_percentage,home_away_wins,AverageHomeWinOdd,HomeTeamid,home_total_goals_for,HomeTeamName,away_failed_to_score_home,home_goals_for_minute_91_105_percentage,AwayTeamName,away_goals_for_minute_46_60_percentage,home_goals_against_minute_76_90,home_total_clean_sheets,away_away_wins,away_goals_against_minute_46_60,home_goals_for_minute_61_75,home_goals_against_minute_91_105,away_away_clean_sheets,home_goals_for_minute_16_30,away_goals_for_minute_16_30,away_goals_for_minute_76_90_percentage,away_goals_for_minute_61_75_percentage,away_goals_against_minute_0_15,home_goals_against_minute_91_105_percentage,home_goals_against_minute_0_15,home_away_clean_sheets,home_home_wins,away_home_goals_for,home_home_goals_for,away_home_goals_against,home_home_draws,away_goals_for_minute_0_15,away_home_wins,home_total_wins,home_total_draws,home_total_loses,away_total_wins,away_total_draws,away_total_loses,away_form_last1games_wins,away_form_last1games_draws,away_form_last1games_loses,away_form_last2games_wins,away_form_last2games_draws,away_form_last2games_loses,away_form_last5games_wins,away_form_last5games_draws,away_form_last5games_loses,away_form_last10games_wins,away_form_last10games_draws,away_form_last10games_loses,home_form_last1games_wins,home_form_last1games_draws,home_form_last1games_loses,home_form_last2games_wins,home_form_last2games_draws,home_form_last2games_loses,home_form_last5games_wins,home_form_last5games_draws,home_form_last5games_loses,home_form_last10games_wins,home_form_last10games_draws,home_form_last10games_loses,total_games,target,predictions
1601,10.00,1.000000,1.631579,0.157895,26/12/2016,D,0.421053,0.777778,16.00,144.0,4.33,12.00,1.87,0.210526,0.400000,16.00,20.00,10.34,0.555556,2.222222,1.89,0.00,0.200000,0.052632,0.210526,24.00,0.250000,1.454545,19.05,17.24,25.00,0.105263,0.454545,0.400000,0.150000,